In [ ]:
# ============================================================================
# CODE POUR ACCÉDER AU DOSSIER PARTAGÉ
# ============================================================================

from google.colab import drive
from pathlib import Path
import os

# 1. Montez VOTRE Drive personnel
print("🔐 Montez votre Drive personnel...")
drive.mount('/content/drive')

# 2. Créez un raccourci (optionnel mais pratique)
print("\n📁 Recherche du dossier partagé 'ISIC_2019_Project'...")

# Méthode 1: Chercher automatiquement
!find '/content/drive' -name "ISIC_2019_Project" -type d 2>/dev/null

# Méthode 2: Ajouter un raccourci manuellement
print("""
💡 Si le dossier n'est pas trouvé automatiquement:

1. Allez sur drive.google.com
2. Dans "Partagés avec moi", trouvez "ISIC_2019_Project"
3. Clic droit → "Ajouter un raccourci vers Drive"
4. Choisissez "Mon Drive"
5. Le dossier apparaîtra dans votre Drive principal
""")

# 3. Attendez que l'utilisateur confirme
input("\n⏳ Avez-vous ajouté le raccourci ? (Appuyez sur Entrée quand c'est fait)")

# 4. Configurer les chemins
DRIVE_ROOT = Path('/content/drive/MyDrive/ISIC_2019_Project')

if not DRIVE_ROOT.exists():
    print(f"\n❌ Dossier non trouvé à: {DRIVE_ROOT}")
    print("\n🔍 Autres emplacements possibles...")

    # Chercher dans tout le Drive
    import subprocess
    result = subprocess.run(['find', '/content/drive', '-name', '*ISIC*', '-type', 'd'],
                          capture_output=True, text=True)
    paths = result.stdout.strip().split('\n')

    if paths and paths[0]:
        print("\n📂 Dossiers trouvés:")
        for i, path in enumerate(paths[:5], 1):
            print(f"  {i}. {path}")

        # Prendre le premier
        DRIVE_ROOT = Path(paths[0])
        print(f"\n✅ Utilisation du chemin: {DRIVE_ROOT}")
    else:
        print("\n⚠️  Aucun dossier ISIC trouvé.")
        print("Veuillez entrer le chemin manuellement:")
        custom_path = input("Chemin: ")
        DRIVE_ROOT = Path(custom_path)

# 5. Vérifier la structure
print("\n" + "="*60)
print("📋 Vérification de la structure...")
print("="*60)

DATA_DIR = DRIVE_ROOT / 'data'
IMG_DIR = DATA_DIR / 'ISIC_2019_Training_Input' / 'ISIC_2019_Training_Input'
GT_PATH = DATA_DIR / 'ISIC_2019_Training_GroundTruth.csv'
META_PATH = DATA_DIR / 'ISIC_2019_Training_Metadata.csv'

print(f"📁 Racine: {DRIVE_ROOT}")
print(f"📊 Data dir: {'✅ Existe' if DATA_DIR.exists() else '❌ Manquant'}")
print(f"🖼️ Images: {'✅ Existe' if IMG_DIR.exists() else '❌ Manquant'}")
print(f"📄 Ground Truth: {'✅ Existe' if GT_PATH.exists() else '❌ Manquant'}")
print(f"📋 Metadata: {'✅ Existe' if META_PATH.exists() else '❌ Manquant'}")

# 6. Lister le contenu pour vérifier
if DRIVE_ROOT.exists():
    print(f"\n📁 Contenu de {DRIVE_ROOT}:")
    !ls -la "{DRIVE_ROOT}"

    if DATA_DIR.exists():
        print(f"\n📊 Contenu de data/:")
        !ls -la "{DATA_DIR}"

print("\n✅ Configuration terminée!")
print("Vous pouvez maintenant exécuter votre code EDA.")

In [ ]:
# MONTER LE DRIVE
from google.colab import drive

print("🔐 Montez votre Drive personnel...")
drive.mount('/content/drive')

print("\n✅ Drive monté avec succès!")
print("📁 Vérification du dossier ISIC_2019_Project...")

from pathlib import Path
DRIVE_ROOT = Path('/content/drive/MyDrive/ISIC_2019_Project')

if DRIVE_ROOT.exists():
    print(f"✅ Dossier trouvé: {DRIVE_ROOT}")
    print("Contenu:")
    !ls -la "{DRIVE_ROOT}"
else:
    print("❌ Dossier non trouvé")

In [ ]:
# ============================================================================
# ISIC 2019 - EDA COMPLÈTE
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime
from PIL import Image
from scipy import stats
import warnings
import shutil
from google.colab import files
from tqdm import tqdm
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# CONFIGURATION - CHEMINS (déjà vérifiés)
# ============================================================================

print("=" * 80)
print("  ISIC 2019 - EDA avec Stockage Local")
print("=" * 80)

# Vos chemins (déjà vérifiés)
DRIVE_ROOT = Path('/content/drive/MyDrive/ISIC_2019_Project')
DATA_DIR = DRIVE_ROOT / 'data'
IMG_DIR = DATA_DIR / 'ISIC_2019_Training_Input' / 'ISIC_2019_Training_Input'
GT_PATH = DATA_DIR / 'ISIC_2019_Training_GroundTruth.csv'
META_PATH = DATA_DIR / 'ISIC_2019_Training_Metadata.csv'

print(f"\n✅ Données lues depuis Drive: {DATA_DIR}")

# Résultats EDA dans STOCKAGE LOCAL Colab ✅
EDA_DIR = Path('/content/eda_results')  # ❌ PAS dans Drive!
EDA_DIR.mkdir(exist_ok=True)

# Structure
for subdir in ['plots', 'statistics', 'samples', 'reports', 'correlations', 'quality_checks']:
    (EDA_DIR / subdir).mkdir(exist_ok=True)

print(f"✅ Résultats sauvegardés localement: {EDA_DIR}")
print(f"✅ À la fin: Téléchargement ZIP automatique")
print("=" * 80)

# Configuration des classes
CLASS_INFO = {
    'MEL': 'Melanoma', 'NV': 'Nevus', 'BCC': 'Basal cell carcinoma',
    'AK': 'Actinic keratosis', 'BKL': 'Benign keratosis',
    'DF': 'Dermatofibroma', 'VASC': 'Vascular lesion', 'SCC': 'Squamous cell carcinoma'
}
classes = list(CLASS_INFO.keys())

# Configuration pour économiser l'espace
DPI_RESOLUTION = 150  # Au lieu de 300
SAVE_SAMPLE_IMAGES = True  # Mettre False pour économiser
MAX_IMAGES_TO_ANALYZE = 2000  # Réduire si besoin

# ============================================================================
# 1. CHARGEMENT DES DONNÉES
# ============================================================================

print("\n📥 Chargement des données...")
gt_df = pd.read_csv(GT_PATH)
meta_df = pd.read_csv(META_PATH)
df = pd.merge(gt_df, meta_df, on='image', how='inner')

print(f"✅ {len(df):,} échantillons chargés")

# Sauvegarder info de base
basic_info = {
    'dataset_name': 'ISIC 2019',
    'total_samples': int(len(df)),
    'num_classes': len(classes),
    'classes': classes,
    'analysis_date': datetime.now().isoformat()
}

with open(EDA_DIR / 'statistics' / '00_dataset_info.json', 'w') as f:
    json.dump(basic_info, f, indent=2)

# ============================================================================
# 2. DISTRIBUTION DES CLASSES
# ============================================================================

print("\n" + "=" * 80)
print("  1. DISTRIBUTION DES CLASSES")
print("=" * 80)

class_counts = gt_df[classes].sum().sort_values(ascending=False)

for cls, count in class_counts.items():
    pct = (count / len(gt_df)) * 100
    print(f"{cls:6s}: {count:6,} ({pct:5.2f}%)")

# Calculs
max_class = class_counts.max()
min_class = class_counts.min()
imbalance_ratio = max_class / min_class
class_weights = {cls: len(gt_df) / (len(classes) * count)
                 for cls, count in class_counts.items()}

print(f"\n⚠️  Ratio déséquilibre: {imbalance_ratio:.1f}:1")

# Sauvegarder
class_stats = {
    'counts': {cls: int(count) for cls, count in class_counts.items()},
    'percentages': {cls: float(count / len(gt_df) * 100) for cls, count in class_counts.items()},
    'imbalance_ratio': float(imbalance_ratio),
    'recommended_weights': {cls: float(w) for cls, w in class_weights.items()}
}

with open(EDA_DIR / 'statistics' / '01_class_distribution.json', 'w') as f:
    json.dump(class_stats, f, indent=2)

pd.DataFrame(class_stats['counts'].items(), columns=['Class', 'Count']).to_csv(
    EDA_DIR / 'statistics' / '01_class_distribution.csv', index=False
)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

class_counts.plot(kind='bar', ax=axes[0], color='steelblue', edgecolor='black')
axes[0].set_title('Distribution des Classes', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Classe')
axes[0].set_ylabel('Nombre')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(alpha=0.3)

for i, (cls, count) in enumerate(class_counts.items()):
    axes[0].text(i, count + 200, f'{count:,}', ha='center', fontweight='bold', fontsize=8)

axes[1].pie(class_counts, labels=class_counts.index, autopct='%1.1f%%',
           startangle=90, colors=plt.cm.Set3(range(len(classes))))
axes[1].set_title('Proportion des Classes', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig(EDA_DIR / 'plots' / '01_class_distribution.png', dpi=DPI_RESOLUTION, bbox_inches='tight')
print(f"💾 Sauvegardé: 01_class_distribution.png")
plt.show()
plt.close()

# ============================================================================
# 3. MÉTADONNÉES
# ============================================================================

print("\n" + "=" * 80)
print("  2. MÉTADONNÉES CLINIQUES")
print("=" * 80)

metadata_stats = {}

if 'age_approx' in meta_df.columns:
    age_data = meta_df['age_approx'].dropna()
    metadata_stats['age'] = {
        'mean': float(age_data.mean()),
        'median': float(age_data.median()),
        'std': float(age_data.std()),
        'min': float(age_data.min()),
        'max': float(age_data.max()),
        'missing_pct': float(meta_df['age_approx'].isna().sum() / len(meta_df) * 100)
    }
    print(f"👤 Âge: {metadata_stats['age']['mean']:.1f} ± {metadata_stats['age']['std']:.1f} ans")

if 'sex' in meta_df.columns:
    sex_counts = meta_df['sex'].value_counts()
    metadata_stats['sex'] = {
        'distribution': {str(k): int(v) for k, v in sex_counts.items()},
        'missing_pct': float(meta_df['sex'].isna().sum() / len(meta_df) * 100)
    }
    print(f"⚥ Sexe: {dict(sex_counts)}")

if 'anatom_site_general' in meta_df.columns:
    site_counts = meta_df['anatom_site_general'].value_counts()
    metadata_stats['anatomical_site'] = {
        'unique_sites': int(len(site_counts)),
        'top_5': {str(k): int(v) for k, v in site_counts.head(5).items()}
    }
    print(f"📍 Sites: {metadata_stats['anatomical_site']['unique_sites']} uniques")

with open(EDA_DIR / 'statistics' / '02_metadata_stats.json', 'w') as f:
    json.dump(metadata_stats, f, indent=2)

# Visualisation métadonnées
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

if 'age_approx' in meta_df.columns:
    age_clean = meta_df['age_approx'].dropna()
    axes[0, 0].hist(age_clean, bins=40, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(age_clean.mean(), color='red', linestyle='--', linewidth=2,
                      label=f'Moyenne: {age_clean.mean():.1f}')
    axes[0, 0].set_title('Distribution Âge', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Âge (années)')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)

if 'sex' in meta_df.columns:
    sex_counts.plot(kind='bar', ax=axes[0, 1], color=['lightcoral', 'lightblue'],
                   edgecolor='black')
    axes[0, 1].set_title('Distribution Sexe', fontsize=12, fontweight='bold')
    axes[0, 1].tick_params(axis='x', rotation=0)
    axes[0, 1].grid(alpha=0.3)

if 'anatom_site_general' in meta_df.columns:
    site_counts.head(8).plot(kind='barh', ax=axes[1, 0], color='lightgreen',
                            edgecolor='black')
    axes[1, 0].set_title('Top 8 Sites Anatomiques', fontsize=12, fontweight='bold')
    axes[1, 0].grid(alpha=0.3)

axes[1, 1].axis('off')
axes[1, 1].text(0.5, 0.5, 'Métadonnées\nAnalysées', ha='center', va='center',
               fontsize=16, fontweight='bold')

plt.tight_layout()
plt.savefig(EDA_DIR / 'plots' / '02_metadata.png', dpi=DPI_RESOLUTION, bbox_inches='tight')
print(f"💾 Sauvegardé: 02_metadata.png")
plt.show()
plt.close()

# ============================================================================
# 4. PROPRIÉTÉS DES IMAGES
# ============================================================================

print("\n" + "=" * 80)
print("  3. PROPRIÉTÉS DES IMAGES")
print("=" * 80)

sample_images = gt_df.sample(n=min(MAX_IMAGES_TO_ANALYZE, len(gt_df)), random_state=42)

widths, heights, aspect_ratios, file_sizes = [], [], [], []

print("📊 Analyse des propriétés d'images...")
for idx, row in tqdm(sample_images.iterrows(), total=len(sample_images), desc="Analyse"):
    img_path = IMG_DIR / f"{row['image']}.jpg"

    if img_path.exists():
        try:
            file_sizes.append(img_path.stat().st_size / 1024)
            img = Image.open(img_path)
            w, h = img.size
            widths.append(w)
            heights.append(h)
            aspect_ratios.append(w / h)
        except:
            continue

image_stats = {
    'sample_size': len(widths),
    'width': {'mean': float(np.mean(widths)), 'std': float(np.std(widths)),
              'min': int(np.min(widths)), 'max': int(np.max(widths))},
    'height': {'mean': float(np.mean(heights)), 'std': float(np.std(heights)),
               'min': int(np.min(heights)), 'max': int(np.max(heights))},
    'aspect_ratio': {'mean': float(np.mean(aspect_ratios)), 'std': float(np.std(aspect_ratios))},
    'file_size_kb': {'mean': float(np.mean(file_sizes)), 'std': float(np.std(file_sizes))}
}

print(f"📏 {len(widths):,} images analysées:")
print(f"   Dimensions: {image_stats['width']['mean']:.0f}×{image_stats['height']['mean']:.0f} px")
print(f"   Taille: {image_stats['file_size_kb']['mean']:.1f} KB")

with open(EDA_DIR / 'statistics' / '03_image_properties.json', 'w') as f:
    json.dump(image_stats, f, indent=2)

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].hist(widths, bins=40, color='coral', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(np.mean(widths), color='red', linestyle='--', linewidth=2)
axes[0, 0].set_title('Distribution Largeurs', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Pixels')
axes[0, 0].grid(alpha=0.3)

axes[0, 1].hist(heights, bins=40, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 1].axvline(np.mean(heights), color='red', linestyle='--', linewidth=2)
axes[0, 1].set_title('Distribution Hauteurs', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Pixels')
axes[0, 1].grid(alpha=0.3)

axes[1, 0].hist(aspect_ratios, bins=40, color='lightgreen', edgecolor='black', alpha=0.7)
axes[1, 0].axvline(1.0, color='purple', linestyle=':', linewidth=2, label='Carré (1:1)')
axes[1, 0].set_title('Ratios d\'Aspect', fontsize=12, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

axes[1, 1].hist(file_sizes, bins=40, color='lightyellow', edgecolor='black', alpha=0.7)
axes[1, 1].set_title('Tailles Fichiers', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('KB')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(EDA_DIR / 'plots' / '03_image_properties.png', dpi=DPI_RESOLUTION, bbox_inches='tight')
print(f"💾 Sauvegardé: 03_image_properties.png")
plt.show()
plt.close()

# ============================================================================
# 5. ÉCHANTILLONS D'IMAGES (optionnel)
# ============================================================================

if SAVE_SAMPLE_IMAGES:
    print("\n" + "=" * 80)
    print("  4. ÉCHANTILLONS D'IMAGES")
    print("=" * 80)

    # Grille complète
    fig, axes = plt.subplots(8, 5, figsize=(16, 24))
    fig.suptitle('Échantillons par Classe', fontsize=14, fontweight='bold')

    for i, cls in enumerate(classes):
        samples = gt_df[gt_df[cls] == 1].sample(n=min(5, (gt_df[cls] == 1).sum()),
                                                random_state=42)

        for j, (idx, row) in enumerate(samples.iterrows()):
            img_path = IMG_DIR / f"{row['image']}.jpg"

            if img_path.exists():
                try:
                    img = Image.open(img_path)
                    axes[i, j].imshow(img)
                    axes[i, j].axis('off')
                    if j == 0:
                        axes[i, j].set_title(f"{cls}\n({class_counts[cls]:,})",
                                           fontsize=9, fontweight='bold', loc='left')
                except:
                    axes[i, j].axis('off')
            else:
                axes[i, j].axis('off')
                axes[i, j].text(0.5, 0.5, 'Image\nnon trouvée',
                               ha='center', va='center', fontsize=8)

    plt.tight_layout()
    plt.savefig(EDA_DIR / 'plots' / '04_samples_grid.png', dpi=DPI_RESOLUTION, bbox_inches='tight')
    print(f"💾 Sauvegardé: 04_samples_grid.png")
    plt.show()
    plt.close()

    # Sauvegarder 2 échantillons par classe (léger)
    print("\n📸 Sauvegarde échantillons individuels...")
    for cls in classes:
        samples = gt_df[gt_df[cls] == 1].sample(n=min(2, (gt_df[cls] == 1).sum()),
                                               random_state=42)
        saved_count = 0
        for idx, (_, row) in enumerate(samples.iterrows(), 1):
            img_path = IMG_DIR / f"{row['image']}.jpg"
            if img_path.exists():
                try:
                    img = Image.open(img_path)
                    # Réduire taille pour économiser
                    img.thumbnail((400, 400))
                    img.save(EDA_DIR / 'samples' / f'{cls}_sample_{idx}.jpg', quality=85)
                    saved_count += 1
                except:
                    continue
        print(f"   ✅ {cls}: {saved_count} échantillons sauvegardés")

# ============================================================================
# 6. RAPPORT MARKDOWN
# ============================================================================

print("\n" + "=" * 80)
print("  5. GÉNÉRATION DU RAPPORT")
print("=" * 80)

report = f"""# ISIC 2019 - Rapport EDA

**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Échantillons:** {len(df):,}
**Classes:** {len(classes)}

## Distribution des Classes

| Classe | Count | % | Poids |
|--------|-------|---|-------|
"""

for cls in classes:
    count = class_counts[cls]
    pct = count / len(gt_df) * 100
    weight = class_weights[cls]
    report += f"| {cls} | {count:,} | {pct:.2f}% | {weight:.3f} |\n"

report += f"""

**Ratio déséquilibre:** {imbalance_ratio:.1f}:1

## Métadonnées

- **Âge moyen:** {metadata_stats.get('age', {}).get('mean', 0):.1f} ans
- **Sites anatomiques:** {metadata_stats.get('anatomical_site', {}).get('unique_sites', 0)} uniques

## Images

- **Dimensions moyennes:** {image_stats['width']['mean']:.0f}×{image_stats['height']['mean']:.0f} px
- **Ratio d'aspect:** {image_stats['aspect_ratio']['mean']:.2f}
- **Taille fichier:** {image_stats['file_size_kb']['mean']:.1f} KB

## Recommandations

1. ⚠️ **Class weighting obligatoire** (ratio {imbalance_ratio:.0f}:1)
2. 🖼️ **Resize à 224×224** pixels
3. 🎨 **Data augmentation** pour classes minoritaires
4. 📊 **Metrics:** AUC-ROC, Balanced Accuracy

---

Généré automatiquement - Prêt pour GitHub
"""

with open(EDA_DIR / 'reports' / 'EDA_REPORT.md', 'w', encoding='utf-8') as f:
    f.write(report)

# README
readme = f"""# EDA Results - ISIC 2019

Résultats de l'analyse exploratoire complète.

## Contenu

- `plots/` - Visualisations
- `statistics/` - Stats JSON/CSV
- `samples/` - Images échantillons
- `reports/` - Rapports Markdown

## Insights Clés

- **Déséquilibre:** {imbalance_ratio:.1f}:1 → Class weighting nécessaire
- **Images:** {image_stats['width']['mean']:.0f}×{image_stats['height']['mean']:.0f} px moyenne
- **Âge moyen:** {metadata_stats.get('age', {}).get('mean', 0):.1f} ans

Voir `EDA_REPORT.md` pour détails complets.
"""

with open(EDA_DIR / 'README.md', 'w', encoding='utf-8') as f:
    f.write(readme)

print(" Rapports générés!")

# ============================================================================
# 7. CRÉATION DU ZIP ET TÉLÉCHARGEMENT
# ============================================================================

print("\n" + "=" * 80)
print("  6. CRÉATION DU ZIP")
print("=" * 80)

zip_path = '/content/eda_results_complete'
shutil.make_archive(zip_path, 'zip', EDA_DIR)

# Statistiques du ZIP
zip_size_mb = Path(f'{zip_path}.zip').stat().st_size / (1024 * 1024)

print(f"\n📦 ZIP créé: eda_results_complete.zip ({zip_size_mb:.1f} MB)")

# Compter les fichiers
import os
plot_count = len([f for f in os.listdir(EDA_DIR / 'plots') if f.endswith('.png')])
stats_count = len([f for f in os.listdir(EDA_DIR / 'statistics')])
samples_count = len([f for f in os.listdir(EDA_DIR / 'samples') if f.endswith('.jpg')])
reports_count = len([f for f in os.listdir(EDA_DIR / 'reports')])

print(f"\n Contenu:")
print(f"   • {plot_count} visualisations")
print(f"   • {stats_count} fichiers stats")
print(f"   • {samples_count} échantillons")
print(f"   • {reports_count} rapports")

print("\n Téléchargement automatique du ZIP...")
files.download(f'{zip_path}.zip')

print("\n" + "=" * 80)
print("  EDA TERMINÉE!")
print("=" * 80)
print(f"\n Prochaines étapes:")
print(f"   1. Le ZIP ({zip_size_mb:.1f} MB) a été téléchargé automatiquement")
print(f"   2. Extraire le ZIP localement sur votre machine")
print(f"   3. Copier le dossier eda_results/ dans votre repo GitHub")
print(f"   4. Commit & push vers GitHub")
print(f"   5. Les données originales restent dans le Drive partagé")
print(f"\n Taille optimisée pour GitHub")
print("=" * 80)

In [ ]:


print("\n🔍 Recherche du bon chemin d'images...")

# Option 1: Essayer différents chemins
possible_img_paths = [
    DATA_DIR / 'ISIC_2019_Training_Input' / 'ISIC_2019_Training_Input',
    DATA_DIR / 'ISIC_2019_Training_Input',
    DATA_DIR / 'images',
    DATA_DIR / 'train',
    DRIVE_ROOT / 'images'
]

IMG_DIR = None
for test_path in possible_img_paths:
    if test_path.exists():
        # Vérifier qu'il y a des fichiers .jpg
        jpg_files = list(test_path.glob("*.jpg"))
        if jpg_files:
            IMG_DIR = test_path
            print(f"✅ Images trouvées dans: {test_path}")
            print(f"   {len(jpg_files):,} fichiers .jpg trouvés")
            break

if IMG_DIR is None:
    print("❌ Aucun dossier d'images trouvé avec des .jpg")
    print("\n🔎 Recherche dans tout le dataset...")
    !find "{DATA_DIR}" -name "*.jpg" -type f 2>/dev/null | head -5

    # Demander le chemin manuellement
    print("\n🎯 Entrez le chemin COMPLET vers le dossier d'images:")
    custom_path = input("Chemin: ")
    IMG_DIR = Path(custom_path)

# Vérification finale
if IMG_DIR.exists():
    image_count = len(list(IMG_DIR.glob("*.jpg")))
    print(f"\n📊 Dossier images confirmé: {IMG_DIR}")
    print(f"📸 Nombre d'images .jpg: {image_count:,}")
else:
    print(f"❌ Chemin invalide: {IMG_DIR}")
    raise FileNotFoundError("Dossier d'images non trouvé")

In [ ]:
# ============================================================================
# DIAGNOSTIC DES NOMS D'IMAGES
# ============================================================================

print("🔍 Diagnostic des correspondances image/CSV...")
print("="*60)

# 1. Voir les premiers noms dans le CSV
print("\n📄 Noms dans Ground Truth CSV (premiers 5):")
print(gt_df['image'].head().tolist())

# 2. Voir les premiers fichiers .jpg
print("\n🖼️  Fichiers .jpg dans le dossier (premiers 5):")
jpg_files = list(IMG_DIR.glob("*.jpg"))
print([f.name for f in jpg_files[:5]])

# 3. Vérifier la correspondance
print("\n🔗 Test de correspondance...")
sample_image_name = gt_df['image'].iloc[0]
expected_path = IMG_DIR / f"{sample_image_name}.jpg"
print(f"Premier nom CSV: {sample_image_name}")
print(f"Chemin attendu: {expected_path}")
print(f"Existe? {expected_path.exists()}")

# 4. Vérifier les extensions
print("\n📋 Analyse des extensions dans le CSV:")
# Regarder s'il y a déjà .jpg dans les noms
has_extension = gt_df['image'].str.endswith('.jpg').any()
print(f"Les noms contiennent déjà .jpg? {has_extension}")

if has_extension:
    print("⚠️  Les noms dans le CSV ont déjà l'extension .jpg")
    # Compter combien
    with_extension = gt_df['image'].str.endswith('.jpg').sum()
    print(f"   {with_extension}/{len(gt_df)} noms avec .jpg")

    # Vérifier un nom spécifique
    sample_with_ext = gt_df[gt_df['image'].str.endswith('.jpg')]['image'].iloc[0] if with_extension > 0 else None
    if sample_with_ext:
        test_path = IMG_DIR / sample_with_ext
        print(f"   Exemple: {sample_with_ext} → {test_path.exists()}")

In [ ]:
# ============================================================================
# ANALYSE DES IMAGES
# ============================================================================

print("🔍 Analyse corrigée de la couverture des images...")
print("="*60)

# 1. Analyser les formats de noms dans le CSV
csv_ids = gt_df['image'].tolist()
print(f"\n📄 Formats des noms dans le CSV (10 premiers):")
for i, name in enumerate(csv_ids[:10]):
    print(f"  {i+1:2}. {name}")

# 2. Extraire les IDs numériques (gérer les noms non standard)
csv_ids_numeric = []
non_numeric_ids = []

for id_str in csv_ids:
    # Essayer d'extraire les chiffres
    import re
    match = re.search(r'(\d+)', id_str)
    if match:
        csv_ids_numeric.append(int(match.group(1)))
    else:
        non_numeric_ids.append(id_str)

print(f"\n📊 Analyse des IDs:")
print(f"   IDs numériques extraits: {len(csv_ids_numeric):,}")
print(f"   IDs non numériques: {len(non_numeric_ids):,}")
if non_numeric_ids:
    print(f"   Exemples d'IDs non numériques: {non_numeric_ids[:5]}")

# 3. IDs dans les fichiers JPG
jpg_files = list(IMG_DIR.glob("*.jpg"))
print(f"\n🖼️  Fichiers .jpg trouvés: {len(jpg_files):,}")

# Analyser les noms de fichiers
jpg_ids_numeric = []
jpg_non_numeric = []

for f in jpg_files:
    stem = f.stem
    match = re.search(r'(\d+)', stem)
    if match:
        jpg_ids_numeric.append(int(match.group(1)))
    else:
        jpg_non_numeric.append(stem)

print(f"\n📊 Analyse des fichiers:")
print(f"   IDs numériques: {len(jpg_ids_numeric):,}")
print(f"   IDs non numériques: {len(jpg_non_numeric):,}")

# 4. Statistiques si nous avons des IDs numériques
if csv_ids_numeric and jpg_ids_numeric:
    min_csv_id = min(csv_ids_numeric)
    max_csv_id = max(csv_ids_numeric)
    min_jpg_id = min(jpg_ids_numeric)
    max_jpg_id = max(jpg_ids_numeric)

    print(f"\n📈 Plages d'IDs:")
    print(f"   CSV: ISIC_{min_csv_id:07d} à ISIC_{max_csv_id:07d}")
    print(f"   JPG: ISIC_{min_jpg_id:07d} à ISIC_{max_jpg_id:07d}")

    # Calculer le chevauchement
    csv_set = set(csv_ids_numeric)
    jpg_set = set(jpg_ids_numeric)
    overlap = csv_set.intersection(jpg_set)

    print(f"\n🔗 Chevauchement:")
    print(f"   IDs communs: {len(overlap):,}")
    print(f"   Coverage: {len(overlap)/len(csv_set)*100:.1f}%")

    # Images manquantes
    missing = csv_set - jpg_set
    if missing:
        print(f"\n❌ IDs manquants dans les fichiers JPG:")
        print(f"   Nombre: {len(missing):,}")
        print(f"   Exemples: {sorted(list(missing))[:5]}")
else:
    print("\n⚠️  Impossible d'analyser les plages (IDs non numériques)")

# 5. VÉRIFICATION DIRECTE DES CORRESPONDANCES
print("\n" + "="*60)
print("✅ VÉRIFICATION DIRECTE DES CORRESPONDANCES")
print("="*60)

# Fonction améliorée pour trouver les images
def find_image_path_v2(image_name):
    """
    Trouve le chemin d'une image avec plusieurs stratégies
    """
    # Stratégie 1: Nom exact + .jpg
    path1 = IMG_DIR / f"{image_name}.jpg"
    if path1.exists():
        return path1

    # Stratégie 2: Nom exact (déjà avec .jpg)
    if image_name.endswith('.jpg'):
        path2 = IMG_DIR / image_name
        if path2.exists():
            return path2

    # Stratégie 3: Extraire l'ID numérique et chercher
    match = re.search(r'(\d+)', image_name)
    if match:
        num_id = match.group(1)
        # Chercher tous les fichiers contenant cet ID
        for f in IMG_DIR.glob(f"*{num_id}*.jpg"):
            return f

    # Stratégie 4: Chercher par pattern
    patterns = [
        f"*{image_name}*.jpg",
        f"*{image_name.replace('_', '*')}*.jpg",
    ]

    for pattern in patterns:
        matches = list(IMG_DIR.glob(pattern))
        if matches:
            return matches[0]

    return None

# Tester avec quelques exemples
print("\n🧪 Test de correspondance (10 premiers):")
test_samples = gt_df['image'].head(10).tolist()

found_count = 0
for i, img_name in enumerate(test_samples):
    path = find_image_path_v2(img_name)
    status = "✅" if path and path.exists() else "❌"
    print(f"  {i+1:2}. {img_name:30} → {status}")
    if path and path.exists():
        found_count += 1

print(f"\n📊 Taux de succès: {found_count}/{len(test_samples)} ({found_count/len(test_samples)*100:.0f}%)")

# 6. ANALYSE COMPLÈTE DE LA DISPONIBILITÉ
print("\n" + "="*60)
print("📊 ANALYSE COMPLÈTE DE DISPONIBILITÉ")
print("="*60)

# Analyser un échantillon pour aller plus vite
SAMPLE_SIZE = min(1000, len(gt_df))
sample_indices = np.random.choice(len(gt_df), SAMPLE_SIZE, replace=False)
sample_images = gt_df['image'].iloc[sample_indices].tolist()

print(f"Analyse d'un échantillon de {SAMPLE_SIZE} images...")
available_count = 0

for img_name in tqdm(sample_images, desc="Vérification"):
    path = find_image_path_v2(img_name)
    if path and path.exists():
        available_count += 1

availability_rate = available_count / SAMPLE_SIZE

print(f"\n📈 Résultats:")
print(f"   Échantillon analysé: {SAMPLE_SIZE}")
print(f"   Images disponibles: {available_count}")
print(f"   Taux de disponibilité: {availability_rate*100:.1f}%")
print(f"   Estimation totale disponible: {int(availability_rate * len(gt_df)):,}")

# 7. SAUVEGARDER L'ANALYSE
print("\n" + "="*60)
print("💾 SAUVEGARDE DES RÉSULTATS")
print("="*60)

availability_stats = {
    'total_in_csv': int(len(gt_df)),
    'sample_analyzed': int(SAMPLE_SIZE),
    'available_in_sample': int(available_count),
    'availability_rate': float(availability_rate),
    'estimated_total_available': int(availability_rate * len(gt_df)),
    'analysis_date': datetime.now().isoformat(),
    'notes': "Certains noms d'images contiennent '_downsampled'"
}

# Sauvegarder
with open(EDA_DIR / 'statistics' / '00_availability_analysis.json', 'w') as f:
    json.dump(availability_stats, f, indent=2)

print(f"✅ Analyse sauvegardée dans: 00_availability_analysis.json")
print(f"\n📋 Résumé:")
print(f"   Vous avez environ {availability_stats['estimated_total_available']:,} images disponibles")
print(f"   sur {availability_stats['total_in_csv']:,} au total ({availability_rate*100:.1f}%)")

print("\n" + "="*80)
print("🎯 RECOMMANDATION: Utiliser uniquement les images disponibles")
print("="*80)

In [ ]:
# ============================================================================
# ISIC 2019 - EDA COMPLÈTE
# ============================================================================

print("=" * 80)
print("  ISIC 2019 - EDA avec 17,757 images disponibles ")
print("=" * 80)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime
from PIL import Image
import warnings
import shutil
from google.colab import files
from tqdm import tqdm
import re
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# CONFIGURATION
# ============================================================================

# Chemins
DRIVE_ROOT = Path('/content/drive/MyDrive/ISIC_2019_Project')
DATA_DIR = DRIVE_ROOT / 'data'
IMG_DIR = DATA_DIR / 'ISIC_2019_Training_Input' / 'ISIC_2019_Training_Input'
GT_PATH = DATA_DIR / 'ISIC_2019_Training_GroundTruth.csv'
META_PATH = DATA_DIR / 'ISIC_2019_Training_Metadata.csv'

# Résultats EDA
EDA_DIR = Path('/content/eda_results')
EDA_DIR.mkdir(exist_ok=True)
for subdir in ['plots', 'statistics', 'samples', 'reports']:
    (EDA_DIR / subdir).mkdir(exist_ok=True)

print(f"\n Données: {DATA_DIR}")
print(f" Résultats: {EDA_DIR}")
print("=" * 80)

# Configuration
CLASS_INFO = {
    'MEL': 'Melanoma', 'NV': 'Nevus', 'BCC': 'Basal cell carcinoma',
    'AK': 'Actinic keratosis', 'BKL': 'Benign keratosis',
    'DF': 'Dermatofibroma', 'VASC': 'Vascular lesion', 'SCC': 'Squamous cell carcinoma'
}
classes = list(CLASS_INFO.keys())

# Optimisation
DPI_RESOLUTION = 150
SAVE_SAMPLE_IMAGES = True  # Maintenant ça marche !
MAX_IMAGES_TO_ANALYZE = 2000  # On peut augmenter car on a les images

# ============================================================================
# 1. CHARGEMENT ET FILTRAGE INTELLIGENT
# ============================================================================

print("\n Chargement et filtrage intelligent des données...")

# Charger les données
gt_df = pd.read_csv(GT_PATH)
meta_df = pd.read_csv(META_PATH)

# Fonction rapide pour vérifier les images
def image_exists_fast(image_name):
    """Vérification rapide de l'existence d'une image"""
    path = IMG_DIR / f"{image_name}.jpg"
    return path.exists()

# Filtrer efficacement
print(" Filtrage des images disponibles...")

# Méthode optimisée : vérifier par lots
batch_size = 1000
available_images = []

for i in tqdm(range(0, len(gt_df), batch_size), desc="Filtrage"):
    batch = gt_df['image'].iloc[i:i+batch_size].tolist()
    for img_name in batch:
        if image_exists_fast(img_name):
            available_images.append(img_name)

print(f"\n Résultat du filtrage:")
print(f"   Total dans CSV: {len(gt_df):,}")
print(f"   Images disponibles: {len(available_images):,}")
print(f"   Taux de disponibilité: {len(available_images)/len(gt_df)*100:.1f}%")

# Filtrer les DataFrames
gt_df_filtered = gt_df[gt_df['image'].isin(available_images)].copy()
meta_df_filtered = meta_df[meta_df['image'].isin(available_images)].copy()

# Fusionner
df = pd.merge(gt_df_filtered, meta_df_filtered, on='image', how='inner')

print(f"\n Dataset filtré créé:")
print(f"   Échantillons: {len(df):,}")
print("=" * 80)

# ============================================================================
# 2. DISTRIBUTION DES CLASSES (FILTRÉE)
# ============================================================================

print("\n" + "=" * 80)
print("  1. DISTRIBUTION DES CLASSES (70.1% du dataset)")
print("=" * 80)

class_counts = df[classes].sum().sort_values(ascending=False)

# Comparaison avec le dataset complet
class_counts_full = gt_df[classes].sum()

print(f"{'Classe':10} {'Disponible':>12} {'Complet':>12} {'%':>6}")
print("-" * 45)

for cls in classes:
    avail = class_counts[cls]
    full = class_counts_full[cls]
    pct = (avail / full * 100) if full > 0 else 0
    print(f"{cls:10} {avail:12,} {full:12,} {pct:6.1f}%")

# Calculs
max_class = class_counts.max()
min_class = class_counts.min()
imbalance_ratio = max_class / min_class
class_weights = {cls: len(df) / (len(classes) * count)
                 for cls, count in class_counts.items()}

print(f"\n  Ratio déséquilibre: {imbalance_ratio:.1f}:1")

# Sauvegarder
class_stats = {
    'total_samples': int(len(df)),
    'coverage_percentage': float(len(df) / len(gt_df) * 100),
    'counts': {cls: int(count) for cls, count in class_counts.items()},
    'percentages': {cls: float(count / len(df) * 100) for cls, count in class_counts.items()},
    'imbalance_ratio': float(imbalance_ratio),
    'recommended_weights': {cls: float(w) for cls, w in class_weights.items()},
    'comparison_with_full': {cls: {
        'available': int(class_counts[cls]),
        'total': int(class_counts_full[cls]),
        'coverage': float(class_counts[cls] / class_counts_full[cls] * 100)
    } for cls in classes}
}

with open(EDA_DIR / 'statistics' / '01_class_distribution_filtered.json', 'w') as f:
    json.dump(class_stats, f, indent=2)

# Visualisation comparée
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribution disponible
class_counts.plot(kind='bar', ax=axes[0], color='steelblue', edgecolor='black')
axes[0].set_title('Distribution (70.1% disponible)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Classe')
axes[0].set_ylabel('Nombre')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(alpha=0.3)

# Coverage par classe
coverage_data = pd.Series({
    cls: class_stats['comparison_with_full'][cls]['coverage']
    for cls in classes
}).sort_values(ascending=False)

coverage_data.plot(kind='bar', ax=axes[1], color='lightcoral', edgecolor='black')
axes[1].set_title('Couverture par Classe (%)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Classe')
axes[1].set_ylabel('Disponibilité (%)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(alpha=0.3)
axes[1].axhline(y=70.1, color='green', linestyle='--', label=f'Moyenne: 70.1%')
axes[1].legend()

plt.tight_layout()
plt.savefig(EDA_DIR / 'plots' / '01_class_distribution_filtered.png', dpi=DPI_RESOLUTION, bbox_inches='tight')
print(f"\n Sauvegardé: 01_class_distribution_filtered.png")
plt.show()
plt.close()

# ============================================================================
# 3. MÉTADONNÉES (FILTRÉES)
# ============================================================================

print("\n" + "=" * 80)
print("  2. MÉTADONNÉES CLINIQUES")
print("=" * 80)

metadata_stats = {}

if 'age_approx' in df.columns:
    age_data = df['age_approx'].dropna()
    metadata_stats['age'] = {
        'mean': float(age_data.mean()),
        'median': float(age_data.median()),
        'std': float(age_data.std()),
        'min': float(age_data.min()),
        'max': float(age_data.max()),
        'missing_pct': float(df['age_approx'].isna().sum() / len(df) * 100)
    }
    print(f"👤 Âge: {metadata_stats['age']['mean']:.1f} ± {metadata_stats['age']['std']:.1f} ans")

if 'sex' in df.columns:
    sex_counts = df['sex'].value_counts()
    metadata_stats['sex'] = {
        'distribution': {str(k): int(v) for k, v in sex_counts.items()},
        'missing_pct': float(df['sex'].isna().sum() / len(df) * 100)
    }
    print(f"⚥ Sexe: {dict(sex_counts)}")

if 'anatom_site_general' in df.columns:
    site_counts = df['anatom_site_general'].value_counts()
    metadata_stats['anatomical_site'] = {
        'unique_sites': int(len(site_counts)),
        'top_5': {str(k): int(v) for k, v in site_counts.head(5).items()}
    }
    print(f" Sites: {metadata_stats['anatomical_site']['unique_sites']} uniques")

with open(EDA_DIR / 'statistics' / '02_metadata_stats_filtered.json', 'w') as f:
    json.dump(metadata_stats, f, indent=2)

# Visualisation métadonnées
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

if 'age_approx' in df.columns:
    age_clean = df['age_approx'].dropna()
    axes[0, 0].hist(age_clean, bins=40, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(age_clean.mean(), color='red', linestyle='--', linewidth=2,
                      label=f'Moyenne: {age_clean.mean():.1f}')
    axes[0, 0].set_title('Distribution Âge', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Âge (années)')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)

if 'sex' in df.columns:
    sex_counts.plot(kind='bar', ax=axes[0, 1], color=['lightcoral', 'lightblue'],
                   edgecolor='black')
    axes[0, 1].set_title('Distribution Sexe', fontsize=12, fontweight='bold')
    axes[0, 1].tick_params(axis='x', rotation=0)
    axes[0, 1].grid(alpha=0.3)

if 'anatom_site_general' in df.columns:
    site_counts.head(8).plot(kind='barh', ax=axes[1, 0], color='lightgreen',
                            edgecolor='black')
    axes[1, 0].set_title('Top 8 Sites Anatomiques', fontsize=12, fontweight='bold')
    axes[1, 0].grid(alpha=0.3)

# Info sur le filtrage
axes[1, 1].axis('off')
axes[1, 1].text(0.5, 0.5, f'Dataset Filtré\n\n{len(df):,} échantillons\n(70.1% du total)\n\nMétadonnées\ncomplètes',
               ha='center', va='center', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(EDA_DIR / 'plots' / '02_metadata_filtered.png', dpi=DPI_RESOLUTION, bbox_inches='tight')
print(f" Sauvegardé: 02_metadata_filtered.png")
plt.show()
plt.close()

# ============================================================================
# 4. PROPRIÉTÉS DES IMAGES (MAINTENANT ÇA MARCHE !)
# ============================================================================

print("\n" + "=" * 80)
print("  3. PROPRIÉTÉS DES IMAGES")
print("=" * 80)

sample_images = df.sample(n=min(MAX_IMAGES_TO_ANALYZE, len(df)), random_state=42)

widths, heights, aspect_ratios, file_sizes = [], [], [], []

print(f" Analyse de {len(sample_images):,} images...")

for idx, row in tqdm(sample_images.iterrows(), total=len(sample_images), desc="Analyse"):
    img_path = IMG_DIR / f"{row['image']}.jpg"

    if img_path.exists():
        try:
            file_sizes.append(img_path.stat().st_size / 1024)
            img = Image.open(img_path)
            w, h = img.size
            widths.append(w)
            heights.append(h)
            aspect_ratios.append(w / h)
        except Exception as e:
            continue

print(f"\n {len(widths):,} images analysées avec succès")

if len(widths) > 0:
    image_stats = {
        'sample_size': len(widths),
        'coverage_percentage': len(sample_images) / len(df) * 100,
        'width': {'mean': float(np.mean(widths)), 'std': float(np.std(widths)),
                  'min': int(np.min(widths)), 'max': int(np.max(widths))},
        'height': {'mean': float(np.mean(heights)), 'std': float(np.std(heights)),
                   'min': int(np.min(heights)), 'max': int(np.max(heights))},
        'aspect_ratio': {'mean': float(np.mean(aspect_ratios)), 'std': float(np.std(aspect_ratios))},
        'file_size_kb': {'mean': float(np.mean(file_sizes)), 'std': float(np.std(file_sizes))}
    }

    print(f"\n📏 Résultats:")
    print(f"   Dimensions: {image_stats['width']['mean']:.0f}×{image_stats['height']['mean']:.0f} px")
    print(f"   Ratio d'aspect: {image_stats['aspect_ratio']['mean']:.2f}")
    print(f"   Taille moyenne: {image_stats['file_size_kb']['mean']:.1f} KB")

    with open(EDA_DIR / 'statistics' / '03_image_properties_filtered.json', 'w') as f:
        json.dump(image_stats, f, indent=2)

    # Visualisation
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    axes[0, 0].hist(widths, bins=40, color='coral', edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(np.mean(widths), color='red', linestyle='--', linewidth=2)
    axes[0, 0].set_title('Distribution Largeurs', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Pixels')
    axes[0, 0].grid(alpha=0.3)

    axes[0, 1].hist(heights, bins=40, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0, 1].axvline(np.mean(heights), color='red', linestyle='--', linewidth=2)
    axes[0, 1].set_title('Distribution Hauteurs', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Pixels')
    axes[0, 1].grid(alpha=0.3)

    axes[1, 0].hist(aspect_ratios, bins=40, color='lightgreen', edgecolor='black', alpha=0.7)
    axes[1, 0].axvline(1.0, color='purple', linestyle=':', linewidth=2, label='Carré (1:1)')
    axes[1, 0].set_title('Ratios d\'Aspect', fontsize=12, fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)

    axes[1, 1].hist(file_sizes, bins=40, color='lightyellow', edgecolor='black', alpha=0.7)
    axes[1, 1].set_title('Tailles Fichiers', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('KB')
    axes[1, 1].grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(EDA_DIR / 'plots' / '03_image_properties_filtered.png', dpi=DPI_RESOLUTION, bbox_inches='tight')
    print(f" Sauvegardé: 03_image_properties_filtered.png")
    plt.show()
    plt.close()
else:
    print(" Aucune image analysée")
    image_stats = {}

# ============================================================================
# 5. ÉCHANTILLONS D'IMAGES (MAINTENANT ÇA MARCHE !)
# ============================================================================

if SAVE_SAMPLE_IMAGES:
    print("\n" + "=" * 80)
    print("  4. ÉCHANTILLONS D'IMAGES")
    print("=" * 80)

    # Grille complète
    fig, axes = plt.subplots(8, 5, figsize=(16, 24))
    fig.suptitle(f'Échantillons par Classe ({len(df):,} images disponibles)',
                fontsize=14, fontweight='bold')

    images_found = 0

    for i, cls in enumerate(classes):
        # Prendre des échantillons de cette classe
        class_samples = df[df[cls] == 1]
        if len(class_samples) > 0:
            samples = class_samples.sample(n=min(5, len(class_samples)), random_state=42)
        else:
            # Si aucune image pour cette classe (dans notre sous-ensemble)
            samples = pd.DataFrame(columns=df.columns)

        for j in range(5):
            if j < len(samples):
                row = samples.iloc[j]
                img_path = IMG_DIR / f"{row['image']}.jpg"

                if img_path.exists():
                    try:
                        img = Image.open(img_path)
                        axes[i, j].imshow(img)
                        axes[i, j].axis('off')
                        if j == 0:
                            axes[i, j].set_title(f"{cls}\n({class_counts[cls]:,})",
                                               fontsize=9, fontweight='bold', loc='left')
                        images_found += 1
                    except:
                        axes[i, j].axis('off')
                        axes[i, j].text(0.5, 0.5, 'Erreur',
                                       ha='center', va='center', fontsize=8)
                else:
                    axes[i, j].axis('off')
                    axes[i, j].text(0.5, 0.5, 'Non trouvé',
                                   ha='center', va='center', fontsize=8)
            else:
                axes[i, j].axis('off')
                axes[i, j].text(0.5, 0.5, 'N/A',
                               ha='center', va='center', fontsize=8)

    plt.tight_layout()
    plt.savefig(EDA_DIR / 'plots' / '04_samples_grid_filtered.png', dpi=DPI_RESOLUTION, bbox_inches='tight')
    print(f" Sauvegardé: 04_samples_grid_filtered.png")
    print(f" Images affichées: {images_found}/40")
    plt.show()
    plt.close()

    # Sauvegarder échantillons individuels
    print("\n Sauvegarde échantillons individuels...")

    for cls in classes:
        class_samples = df[df[cls] == 1]
        if len(class_samples) > 0:
            samples = class_samples.sample(n=min(2, len(class_samples)), random_state=42)
            saved_count = 0

            for idx, (_, row) in enumerate(samples.iterrows(), 1):
                img_path = IMG_DIR / f"{row['image']}.jpg"
                if img_path.exists():
                    try:
                        img = Image.open(img_path)
                        img.thumbnail((400, 400))
                        img.save(EDA_DIR / 'samples' / f'{cls}_sample_{idx}.jpg', quality=85)
                        saved_count += 1
                    except:
                        continue

            print(f"   ✅ {cls}: {saved_count} échantillon(s)")
        else:
            print(f"     {cls}: Aucun échantillon disponible")

# ============================================================================
# 6. RAPPORT MARKDOWN COMPLET
# ============================================================================

print("\n" + "=" * 80)
print("  5. GÉNÉRATION DU RAPPORT")
print("=" * 80)

# Créer rapport détaillé
report = f"""# ISIC 2019 - Rapport EDA (Dataset Partiel)

**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Statut:** Dataset partiel (70.1% disponible)
**Échantillons analysés:** {len(df):,} / {len(gt_df):,}
**Classes:** {len(classes)}

##  Couverture du Dataset

Vous travaillez avec **{len(df):,} images** sur **{len(gt_df):,}** au total.

### Distribution par Classe

| Classe | Disponible | Total | Couverture | Poids recommandé |
|--------|------------|-------|------------|------------------|
"""

for cls in classes:
    avail = class_counts[cls]
    total = class_counts_full[cls]
    coverage = (avail / total * 100) if total > 0 else 0
    weight = class_weights.get(cls, 1.0)
    report += f"| {cls} | {avail:,} | {total:,} | {coverage:.1f}% | {weight:.3f} |\n"

report += f"""

**Ratio déséquilibre:** {imbalance_ratio:.1f}:1
**Couverture moyenne:** {len(df)/len(gt_df)*100:.1f}%

## 🩺 Métadonnées Cliniques

"""

if 'age_approx' in df.columns:
    report += f"- **Âge moyen:** {metadata_stats.get('age', {}).get('mean', 0):.1f} ans\n"

if 'sex' in df.columns:
    sex_dist = metadata_stats.get('sex', {}).get('distribution', {})
    report += f"- **Sexe:** {', '.join([f'{k}: {v}' for k, v in sex_dist.items()])}\n"

if 'anatom_site_general' in df.columns:
    report += f"- **Sites anatomiques:** {metadata_stats.get('anatomical_site', {}).get('unique_sites', 0)} uniques\n"

report += f"""

## 🖼️ Propriétés des Images

"""

if len(widths) > 0:
    report += f"""- **Dimensions moyennes:** {image_stats['width']['mean']:.0f}×{image_stats['height']['mean']:.0f} px
- **Ratio d'aspect:** {image_stats['aspect_ratio']['mean']:.2f}
- **Taille fichier moyenne:** {image_stats['file_size_kb']['mean']:.1f} KB
- **Images analysées:** {image_stats['sample_size']:,} échantillons
"""
else:
    report += "- *Propriétés des images non analysées*\n"

report += f"""

## ⚠️ Limitations

1. **Dataset incomplet** - 70.1% des images seulement
2. **Déséquilibre de classes** - Ratio {imbalance_ratio:.1f}:1
3. **Images manquantes:** {len(gt_df) - len(df):,}

## 🎯 Recommandations pour l'Entraînement

1. **Utiliser les poids de classe** pour compenser le déséquilibre
2. **Data augmentation** intensive pour les classes minoritaires
3. **Évaluer sur un sous-ensemble complet** si possible
4. **Métriques:** AUC-ROC, Balanced Accuracy, F1-Score
5. **Cross-validation** stratifiée recommandée

---

*Généré automatiquement - ISIC 2019 EDA - Dataset partiel (70.1%)*
"""

with open(EDA_DIR / 'reports' / 'EDA_REPORT_FILTERED.md', 'w', encoding='utf-8') as f:
    f.write(report)

print(" Rapport généré: EDA_REPORT_FILTERED.md")

# ============================================================================
# 7. CRÉATION DU ZIP FINAL
# ============================================================================

print("\n" + "=" * 80)
print("  6. CRÉATION DU ZIP FINAL")
print("=" * 80)

zip_path = '/content/eda_results_filtered'
shutil.make_archive(zip_path, 'zip', EDA_DIR)

# Statistiques
zip_size_mb = Path(f'{zip_path}.zip').stat().st_size / (1024 * 1024)

print(f"\n ZIP créé: eda_results_filtered.zip ({zip_size_mb:.1f} MB)")

# Compter les fichiers
import os
plot_count = len([f for f in os.listdir(EDA_DIR / 'plots') if f.endswith('.png')])
stats_count = len([f for f in os.listdir(EDA_DIR / 'statistics')])
samples_count = len([f for f in os.listdir(EDA_DIR / 'samples') if f.endswith('.jpg')])

print(f"\n Contenu du ZIP:")
print(f"   • {plot_count} visualisations")
print(f"   • {stats_count} fichiers de statistiques")
print(f"   • {samples_count} images échantillons")
print(f"   • 2 rapports Markdown")

print("\n⬇ Téléchargement automatique...")
files.download(f'{zip_path}.zip')

print("\n" + "=" * 80)
print("   EDA TERMINÉE AVEC SUCCÈS !")
print("=" * 80)
print(f"\n FÉLICITATIONS ! Votre EDA est complète.")
print(f"\n Prochaines étapes:")
print(f"   1. Le ZIP ({zip_size_mb:.1f} MB) est téléchargé")
print(f"   2. Vous avez analysé {len(df):,} images disponibles")
print(f"   3. Rapport complet généré")
print(f"   4. Prêt pour l'entraînement de modèle !")
print(f"\n Points clés:")
print(f"   • Dataset: {len(df):,} images (70.1% du total)")
print(f"   • Déséquilibre: ratio {imbalance_ratio:.1f}:1")
print(f"   • Recommandation: Utiliser les poids de classe")
print("=" * 80)

In [ ]:
# ============================================================================
# ANALYSES COMPLÉMENTAIRES

print(" AJOUT D'ANALYSES COMPLÉMENTAIRES")
print("="*60)

# Créer les dossiers s'ils n'existent pas
corr_dir = EDA_DIR / 'correlations'
quality_dir = EDA_DIR / 'quality_checks'
corr_dir.mkdir(exist_ok=True)
quality_dir.mkdir(exist_ok=True)

# 1. ANALYSES DE CORRÉLATION
print("\n Analyses de corrélation...")

# Charger les données si nécessaire
if 'df' not in locals():
    # Charger les données filtrées
    import pandas as pd
    from pathlib import Path

    DATA_DIR = Path('/content/drive/MyDrive/ISIC_2019_Project/data')
    GT_PATH = DATA_DIR / 'ISIC_2019_Training_GroundTruth.csv'
    META_PATH = DATA_DIR / 'ISIC_2019_Training_Metadata.csv'

    gt_df = pd.read_csv(GT_PATH)
    meta_df = pd.read_csv(META_PATH)
    df = pd.merge(gt_df, meta_df, on='image', how='inner')

# Corrélations entre variables numériques
if 'age_approx' in df.columns:
    # Matrice de corrélation
    numeric_cols = ['age_approx'] + classes
    corr_matrix = df[numeric_cols].corr()

    # Sauvegarder
    corr_matrix.to_csv(corr_dir / 'correlation_matrix.csv')

    # Visualisation
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
    plt.title('Matrice de Corrélation (Âge ↔ Classes)')
    plt.tight_layout()
    plt.savefig(corr_dir / 'correlation_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()
    plt.close()

    print(f" Matrice de corrélation sauvegardée")

# 2. CONTRÔLES QUALITÉ
print("\n Contrôles qualité des images...")

import random
from PIL import Image, ImageStat

# Analyser quelques images pour la qualité
quality_results = {
    'images_analyzed': 0,
    'avg_brightness': 0,
    'avg_contrast': 0,
    'corrupted_images': []
}

sample_images = df.sample(n=min(100, len(df)), random_state=42)
brightness_values = []
contrast_values = []

for idx, row in sample_images.iterrows():
    img_path = IMG_DIR / f"{row['image']}.jpg"

    if img_path.exists():
        try:
            img = Image.open(img_path)

            # Vérifier si l'image peut être lue
            img.verify()

            # Rouvrir pour analyse
            img = Image.open(img_path)

            # Calculer la luminosité
            stat = ImageStat.Stat(img)
            brightness = sum(stat.mean) / len(stat.mean)
            brightness_values.append(brightness)

            # Calculer le contraste (écart-type)
            if hasattr(stat, 'stddev'):
                contrast = sum(stat.stddev) / len(stat.stddev)
                contrast_values.append(contrast)

            quality_results['images_analyzed'] += 1

        except Exception as e:
            quality_results['corrupted_images'].append({
                'image': row['image'],
                'error': str(e)[:50]
            })

if brightness_values:
    quality_results['avg_brightness'] = sum(brightness_values) / len(brightness_values)
if contrast_values:
    quality_results['avg_contrast'] = sum(contrast_values) / len(contrast_values)

# Sauvegarder les résultats
import json
with open(quality_dir / 'quality_check_results.json', 'w') as f:
    json.dump(quality_results, f, indent=2)

print(f" Contrôles qualité sauvegardés")
print(f"   Images analysées: {quality_results['images_analyzed']}")
print(f"   Images corrompues: {len(quality_results['corrupted_images'])}")

# 3. RÉSUMÉ
print("\n Résumé des ajouts:")
print(f"   • correlations/: {len(list(corr_dir.glob('*')))} fichiers")
print(f"   • quality_checks/: {len(list(quality_dir.glob('*')))} fichiers")

print("\n" + "="*60)
print(" ANALYSES COMPLÉMENTAIRES TERMINÉES")
print("="*60)

In [ ]:
# DIAGNOSTIC DE VOS DONNÉES
print("🔍 Diagnostic des données pour analyse multivariée...")
print("="*60)

# 1. Colonnes numériques
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\n Colonnes numériques: {len(numeric_cols)}")
for col in numeric_cols:
    non_null = df[col].notna().sum()
    print(f"   • {col}: {non_null:,} valeurs non-null")

# 2. Colonnes catégorielles pouvant être encodées
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\n Colonnes catégorielles: {len(categorical_cols)}")
for col in categorical_cols[:5]:  # Montrer les 5 premières
    unique_vals = df[col].nunique()
    print(f"   • {col}: {unique_vals} valeurs uniques")

# 3. Vérifier les classes
print(f"\n Classes: {len(classes)}")
for cls in classes:
    count = df[cls].sum()
    print(f"   • {cls}: {count:,} échantillons")

print("\n" + "="*60)
print(" DIAGNOSTIC TERMINÉ")
print("="*60)


In [ ]:
# ============================================================================
# PRÉPARATION POUR L'ANALYSE AVANCÉE - ISIC 2019
# ============================================================================

print("=" * 80)
print("  🔧 PRÉPARATION POUR L'ANALYSE AVANCÉE")
print("=" * 80)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime
from PIL import Image
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

# ============================================================================
# 1. VÉRIFIER ET CHARGER LES DONNÉES
# ============================================================================

print("\n Chargement des données...")

# Chemins
DRIVE_ROOT = Path('/content/drive/MyDrive/ISIC_2019_Project')
DATA_DIR = DRIVE_ROOT / 'data'
IMG_DIR = DATA_DIR / 'ISIC_2019_Training_Input' / 'ISIC_2019_Training_Input'
GT_PATH = DATA_DIR / 'ISIC_2019_Training_GroundTruth.csv'
META_PATH = DATA_DIR / 'ISIC_2019_Training_Metadata.csv'

# Résultats EDA
EDA_DIR = Path('/content/eda_results')
if not EDA_DIR.exists():
    EDA_DIR = Path('/content/eda_advanced_results')
    EDA_DIR.mkdir(exist_ok=True)

# Créer les sous-dossiers nécessaires
for subdir in ['plots', 'statistics', 'samples', 'reports']:
    (EDA_DIR / subdir).mkdir(exist_ok=True)

# Charger les données
gt_df = pd.read_csv(GT_PATH)
meta_df = pd.read_csv(META_PATH)

# Fusionner
df = pd.merge(gt_df, meta_df, on='image', how='inner')

print(f" Données chargées: {len(df):,} échantillons")

# ============================================================================
# 2. DÉFINIR LES VARIABLES NÉCESSAIRES
# ============================================================================

print("\n Initialisation des variables...")

# Classes
CLASS_INFO = {
    'MEL': 'Melanoma',
    'NV': 'Nevus',
    'BCC': 'Basal cell carcinoma',
    'AK': 'Actinic keratosis',
    'BKL': 'Benign keratosis',
    'DF': 'Dermatofibroma',
    'VASC': 'Vascular lesion',
    'SCC': 'Squamous cell carcinoma'
}
classes = list(CLASS_INFO.keys())

print(f" Classes définies: {len(classes)}")

# ============================================================================
# 3. STATISTIQUES DE BASE POUR L'ANALYSE AVANCÉE
# ============================================================================

print("\n Calcul des statistiques de base...")

# Statistiques de classes
class_counts = df[classes].sum().sort_values(ascending=False)
max_class = class_counts.max()
min_class = class_counts.min()
imbalance_ratio = max_class / min_class

# Créer le dictionnaire class_stats attendu par l'analyse avancée
class_stats = {
    'total_samples': int(len(df)),
    'coverage_percentage': 100.0,  # Vous pouvez ajuster si nécessaire
    'counts': {cls: int(count) for cls, count in class_counts.items()},
    'percentages': {cls: float(count / len(df) * 100) for cls, count in class_counts.items()},
    'imbalance_ratio': float(imbalance_ratio),
    'recommended_weights': {
        cls: float(len(df) / (len(classes) * count))
        for cls, count in class_counts.items()
    }
}

print(f" Statistiques calculées:")
print(f"   • Total échantillons: {len(df):,}")
print(f"   • Ratio déséquilibre: {imbalance_ratio:.1f}:1")

# ============================================================================
# 4. VÉRIFIER LA DISPONIBILITÉ DES IMAGES
# ============================================================================

print("\n Vérification des images...")

# Vérifier quelques images pour s'assurer que le chemin est correct
test_images = df['image'].head(5).tolist()
images_found = sum(1 for img_name in test_images if (IMG_DIR / f"{img_name}.jpg").exists())

print(f" Images trouvées: {images_found}/{len(test_images)}")

if images_found < len(test_images):
    print(f" Attention: Certaines images ne sont pas trouvées")
    print(f"   Vérifiez le chemin: {IMG_DIR}")
else:
    print(f" Chemin des images vérifié: {IMG_DIR}")

# ============================================================================
# 5. RÉSUMÉ ET VARIABLES DISPONIBLES
# ============================================================================

print("\n" + "=" * 80)
print("   PRÉPARATION TERMINÉE - VARIABLES DISPONIBLES")
print("=" * 80)

print("\n Variables initialisées pour l'analyse avancée:")
print(f"   • df: DataFrame avec {len(df):,} lignes et {len(df.columns)} colonnes")
print(f"   • classes: Liste de {len(classes)} classes diagnostiques")
print(f"   • CLASS_INFO: Dictionnaire des noms de classes")
print(f"   • class_stats: Statistiques complètes des classes")
print(f"   • IMG_DIR: {IMG_DIR}")
print(f"   • EDA_DIR: {EDA_DIR}")

print("\n📊 Colonnes disponibles dans df:")
print(f"   {', '.join(df.columns.tolist()[:10])}...")

print("\n🎯 Variables numériques disponibles:")
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"   {', '.join([col for col in numeric_cols if col not in classes][:5])}")

print("\n" + "=" * 80)
print("  🚀 PRÊT POUR L'ANALYSE AVANCÉE !")
print("=" * 80)

print("\n Prochaines étapes:")
print("   1.  Variables initialisées")
print("   2.  Chemins vérifiés")
print("   3.  Exécutez maintenant le script d'analyse avancée")
print("\n   Toutes les variables nécessaires sont maintenant disponibles.")

# ============================================================================
# 6. SAUVEGARDER UN CHECKPOINT
# ============================================================================

print("\n Sauvegarde d'un checkpoint...")

checkpoint_data = {
    'timestamp': datetime.now().isoformat(),
    'total_samples': len(df),
    'classes': classes,
    'class_counts': {cls: int(count) for cls, count in class_counts.items()},
    'paths': {
        'IMG_DIR': str(IMG_DIR),
        'EDA_DIR': str(EDA_DIR),
        'GT_PATH': str(GT_PATH),
        'META_PATH': str(META_PATH)
    },
    'numeric_columns': numeric_cols,
    'ready_for_advanced_analysis': True
}

with open(EDA_DIR / 'checkpoint_advanced_prep.json', 'w') as f:
    json.dump(checkpoint_data, f, indent=2)

print(f" Checkpoint sauvegardé: {EDA_DIR / 'checkpoint_advanced_prep.json'}")

print("\n" + "=" * 80)
print("=" * 80)

In [ ]:
# ============================================================================
# SUITE DE L'ANALYSE EXPLORATOIRE AVANCÉE - ISIC 2019
# ============================================================================

print("=" * 80)
print("  📊 ANALYSE EXPLORATOIRE AVANCÉE - ISIC 2019")
print("=" * 80)

# Importer les bibliothèques nécessaires
import cv2
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score

# #  4. ANALYSE DES COULEURS DES LÉSIONS

# %%
print("\n🎨 ANALYSE DES COULEURS DOMINANTES DES LÉSIONS CUTANÉES")

def extract_skin_lesion_colors(image_path, k=5):
    """
    Extrait les couleurs dominantes d'une lésion cutanée
    """
    try:
        # Charger l'image
        img = cv2.imread(str(image_path))
        if img is None:
            return []

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Redimensionner pour accélérer
        img_resized = cv2.resize(img, (200, 200))

        # Aplatir l'image
        pixels = img_resized.reshape(-1, 3)

        # K-means pour trouver les couleurs dominantes
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(pixels)

        # Compter les labels
        labels = kmeans.labels_
        counts = Counter(labels)

        # Trier par fréquence
        sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

        # Récupérer les couleurs
        colors = []
        for label, count in sorted_counts:
            color = kmeans.cluster_centers_[label].astype(int)
            percentage = (count / len(labels)) * 100
            colors.append((color, percentage))

        return colors
    except Exception as e:
        return []

def analyze_color_distribution_medical(sample_df, img_dir, n_samples=100):
    """
    Analyse la distribution des couleurs sur un échantillon d'images médicales
    """
    print(f"Analyse des couleurs sur {min(n_samples, len(sample_df))} lésions...")

    all_colors = []
    color_percentages = []
    class_labels = []

    sample_data = sample_df.sample(min(n_samples, len(sample_df)), random_state=42)
    images_analyzed = 0

    for idx, row in sample_data.iterrows():
        # Les noms d'image dans ISIC n'ont pas d'extension dans le CSV
        image_name = row['image']

        # Essayer avec .jpg d'abord
        img_path = IMG_DIR / f"{image_name}.jpg"

        # Si pas trouvé, essayer sans extension
        if not img_path.exists():
            img_path = IMG_DIR / image_name

        if img_path.exists():
            try:
                colors = extract_skin_lesion_colors(img_path, k=3)

                if colors:
                    # Récupérer la couleur principale
                    main_color, percentage = colors[0]
                    all_colors.append(main_color)
                    color_percentages.append(percentage)

                    # Trouver la classe
                    for cls in classes:
                        if row[cls] == 1:
                            class_labels.append(cls)
                            break

                    images_analyzed += 1
            except:
                continue

    print(f"✅ {images_analyzed} images analysées avec succès")

    return np.array(all_colors), np.array(color_percentages), class_labels

# Prendre un échantillon pour l'analyse des couleurs
color_sample_size = min(200, len(df))
color_sample_df = df.sample(color_sample_size, random_state=42)

# Analyser les couleurs
dominant_colors, color_percentages, color_class_labels = analyze_color_distribution_medical(
    color_sample_df, IMG_DIR, n_samples=100
)

if len(dominant_colors) > 0:
    # Analyser les canaux de couleur
    print("\n📊 STATISTIQUES DES COULEURS DOMINANTES :")
    print(f"R (Rouge)   : Moyenne = {dominant_colors[:, 0].mean():.1f} ± {dominant_colors[:, 0].std():.1f}")
    print(f"G (Vert)    : Moyenne = {dominant_colors[:, 1].mean():.1f} ± {dominant_colors[:, 1].std():.1f}")
    print(f"B (Bleu)    : Moyenne = {dominant_colors[:, 2].mean():.1f} ± {dominant_colors[:, 2].std():.1f}")
    print(f"Dominance   : Moyenne = {color_percentages.mean():.1f}% (Max = {color_percentages.max():.1f}%)")

    # Visualisation des couleurs dominantes
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))

    # 1. Distribution des canaux
    colors_rgb = ['Rouge', 'Vert', 'Bleu']
    color_codes = ['red', 'green', 'blue']  # Noms en anglais pour matplotlib

    for i, (color_name, color_code, ax) in enumerate(zip(colors_rgb, color_codes, axes[0])):
        ax.hist(dominant_colors[:, i], bins=30, color=color_code, alpha=0.7, edgecolor='black')
        ax.set_title(f'Distribution {color_name}', fontweight='bold')
        ax.set_xlabel('Valeur (0-255)')
        ax.set_ylabel('Fréquence')
        ax.grid(alpha=0.3)

    # 2. Nuage de points RGB coloré par classe
    if color_class_labels and len(set(color_class_labels)) > 1:
        unique_classes = list(set(color_class_labels))

        for cls in unique_classes:
            class_mask = [label == cls for label in color_class_labels]
            if sum(class_mask) > 0:
                axes[1, 0].scatter(
                    dominant_colors[class_mask, 0],
                    dominant_colors[class_mask, 1],
                    label=cls, alpha=0.6, s=30
                )

        axes[1, 0].set_xlabel('Rouge')
        axes[1, 0].set_ylabel('Vert')
        axes[1, 0].set_title('Nuage Rouge-Vert par Classe', fontweight='bold')
        axes[1, 0].legend(fontsize=8)
        axes[1, 0].grid(alpha=0.3)
    else:
        axes[1, 0].scatter(dominant_colors[:, 0], dominant_colors[:, 1], alpha=0.6, s=30)
        axes[1, 0].set_xlabel('Rouge')
        axes[1, 0].set_ylabel('Vert')
        axes[1, 0].set_title('Nuage Rouge-Vert', fontweight='bold')
        axes[1, 0].grid(alpha=0.3)

    # 3. Palette de couleurs dominantes
    if len(dominant_colors) > 10:
        # Créer une palette des 10 couleurs les plus fréquentes
        from collections import Counter

        # Convertir les couleurs en tuples pour les compter
        color_tuples = [tuple(color) for color in dominant_colors]
        color_counts = Counter(color_tuples)
        top_colors = [np.array(color) for color, _ in color_counts.most_common(10)]

        if top_colors:
            axes[0, 2].imshow([top_colors], aspect='auto')
            axes[0, 2].set_title('Top 10 Couleurs', fontweight='bold', fontsize=10)
            axes[0, 2].axis('off')

    # 4. Distribution de dominance
    axes[1, 2].hist(color_percentages, bins=30, color='purple', alpha=0.7, edgecolor='black')
    axes[1, 2].axvline(color_percentages.mean(), color='red', linestyle='--',
                       label=f'Moyenne: {color_percentages.mean():.1f}%')
    axes[1, 2].set_xlabel('% de Dominance')
    axes[1, 2].set_ylabel('Fréquence')
    axes[1, 2].set_title('Distribution de Dominance', fontweight='bold')
    axes[1, 2].legend()
    axes[1, 2].grid(alpha=0.3)

    # 5. Corrélation entre canaux
    if len(dominant_colors) > 10:
        axes[1, 1].scatter(dominant_colors[:, 0], dominant_colors[:, 2], alpha=0.6, s=20, color='blue')
        axes[1, 1].set_xlabel('Rouge')
        axes[1, 1].set_ylabel('Bleu')
        axes[1, 1].set_title('Corrélation Rouge-Bleu', fontweight='bold')
        axes[1, 1].grid(alpha=0.3)
    else:
        axes[1, 1].axis('off')

    # 6. Espace de couleur 3D (simplifié)
    if len(dominant_colors) <= 10:
        # Si peu de couleurs, afficher la palette
        palette = dominant_colors[:10]
        if len(palette) > 0:
            axes[0, 2].imshow([palette], aspect='auto')
            axes[0, 2].set_title('Palette Couleurs', fontweight='bold', fontsize=10)
            axes[0, 2].axis('off')

    plt.suptitle('Analyse des Couleurs Dominantes - Lésions Cutanées ISIC 2019',
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(EDA_DIR / 'plots' / '05_color_analysis_medical.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Analyse avancée : corrélation couleur/classe
    print("\n🔬 ANALYSE COULEUR-CLASSE :")

    if color_class_labels and len(set(color_class_labels)) > 1:
        # Créer un DataFrame pour l'analyse
        color_df = pd.DataFrame(dominant_colors, columns=['R', 'G', 'B'])
        color_df['Class'] = color_class_labels
        color_df['Dominance'] = color_percentages

        # Moyennes par classe
        print("\n🎨 COULEURS MOYENNES PAR CLASSE :")
        class_color_stats = color_df.groupby('Class')[['R', 'G', 'B', 'Dominance']].mean()
        print(class_color_stats.round(1))

        # Sauvegarder les données de couleurs
        color_stats = {
            'overall_stats': {
                'mean_rgb': dominant_colors.mean(axis=0).tolist(),
                'std_rgb': dominant_colors.std(axis=0).tolist(),
                'mean_dominance': float(color_percentages.mean()),
                'samples_analyzed': len(dominant_colors)
            },
            'class_stats': class_color_stats.round(2).to_dict()
        }
    else:
        print("⚠️ Données insuffisantes pour l'analyse par classe")
        color_stats = {
            'overall_stats': {
                'mean_rgb': dominant_colors.mean(axis=0).tolist(),
                'std_rgb': dominant_colors.std(axis=0).tolist(),
                'mean_dominance': float(color_percentages.mean()),
                'samples_analyzed': len(dominant_colors)
            }
        }

    with open(EDA_DIR / 'statistics' / '05_color_analysis.json', 'w') as f:
        json.dump(color_stats, f, indent=2)

    print(f"\n✅ Analyse des couleurs sauvegardée")

else:
    print("⚠️ Aucune couleur analysée - vérifiez les chemins d'images")

# #  5. CORRÉLATIONS ENTRE MÉTADONNÉES ET CLASSES

# %%
print("\n🔗 ANALYSE DES CORRÉLATIONS MÉTADONNÉES-CLASSES")

# Préparer les données pour l'analyse de corrélation
correlation_data = df.copy()

# Convertir les variables catégorielles si présentes
if 'sex' in correlation_data.columns:
    correlation_data['sex_numeric'] = correlation_data['sex'].map({'male': 0, 'female': 1})

if 'anatom_site_general' in correlation_data.columns:
    # Encodage one-hot pour les sites anatomiques
    site_dummies = pd.get_dummies(correlation_data['anatom_site_general'], prefix='site')
    correlation_data = pd.concat([correlation_data, site_dummies], axis=1)

# Colonnes à inclure dans l'analyse de corrélation
correlation_cols = classes.copy()

# Ajouter les métadonnées numériques
if 'age_approx' in correlation_data.columns:
    correlation_cols.append('age_approx')
if 'sex_numeric' in correlation_data.columns:
    correlation_cols.append('sex_numeric')

# Ajouter les sites anatomiques (premiers 5)
site_cols = [col for col in correlation_data.columns if col.startswith('site_')]
correlation_cols.extend(site_cols[:5])

# Calculer la matrice de corrélation
try:
    correlation_matrix = correlation_data[correlation_cols].corr()

    # Extraire uniquement les corrélations entre métadonnées et classes
    metadata_classes_corr = correlation_matrix.loc[classes, [c for c in correlation_cols if c not in classes]]

    # Visualisation avec heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(metadata_classes_corr, annot=True, fmt='.2f', cmap='coolwarm',
                center=0, square=True, linewidths=.5, cbar_kws={"shrink": .8})

    plt.title('Corrélations entre Métadonnées et Classes de Lésions', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(EDA_DIR / 'plots' / '06_metadata_classes_correlation.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Identifier les corrélations significatives
    print("\n📊 CORRÉLATIONS SIGNIFICATIVES (|r| > 0.1) :")
    significant_correlations = []

    for cls in classes:
        for metadata in metadata_classes_corr.columns:
            corr = metadata_classes_corr.loc[cls, metadata]
            if abs(corr) > 0.1:  # Seuil de corrélation faible mais intéressante
                significant_correlations.append((metadata, cls, corr))

    significant_correlations.sort(key=lambda x: abs(x[2]), reverse=True)

    for metadata, cls, corr in significant_correlations[:15]:
        direction = "positive" if corr > 0 else "négative"
        metadata_name = metadata.replace('site_', '').replace('_', ' ').title()
        print(f"  {metadata_name:25} ↔ {cls:8} : r = {corr:.3f} ({direction})")

    # Analyse des corrélations fortes par classe
    print("\n🎯 CORRÉLATIONS FORTES PAR CLASSE (top 3) :")
    for cls in classes:
        cls_correlations = [(m, c) for m, c in metadata_classes_corr.loc[cls].items()]
        cls_correlations.sort(key=lambda x: abs(x[1]), reverse=True)

        if cls_correlations and abs(cls_correlations[0][1]) > 0.1:
            print(f"\n  {cls} ({CLASS_INFO[cls]}):")
            for metadata, corr in cls_correlations[:3]:
                if abs(corr) > 0.05:  # Seuil minimal
                    metadata_name = metadata.replace('site_', '').replace('_', ' ').title()
                    direction = "positive" if corr > 0 else "négative"
                    print(f"    • {metadata_name:25} : r = {corr:.3f} ({direction})")

    # Sauvegarder les corrélations
    correlation_stats = {
        'significant_correlations': [
            {
                'metadata': m,
                'class': c,
                'correlation': float(corr),
                'direction': 'positive' if corr > 0 else 'negative'
            }
            for m, c, corr in significant_correlations if abs(corr) > 0.1
        ],
        'correlation_matrix_shape': metadata_classes_corr.shape,
        'analysis_timestamp': datetime.now().isoformat()
    }

except Exception as e:
    print(f"⚠️ Erreur dans l'analyse de corrélation: {e}")
    correlation_stats = {
        'significant_correlations': [],
        'correlation_matrix_shape': (0, 0),
        'analysis_timestamp': datetime.now().isoformat()
    }

with open(EDA_DIR / 'statistics' / '06_correlation_analysis.json', 'w') as f:
    json.dump(correlation_stats, f, indent=2)

print(f"\n✅ Analyse des corrélations sauvegardée")

# #  6. ANALYSE EN COMPOSANTES PRINCIPALES (PCA) MÉDICALE

# %%
print("\n📊 ANALYSE EN COMPOSANTES PRINCIPALES (PCA) MÉDICALE")

# Préparer les données pour PCA
pca_features = []

# Ajouter l'âge si disponible
if 'age_approx' in df.columns:
    age_data = df['age_approx'].fillna(df['age_approx'].median()).values.reshape(-1, 1)
    pca_features.append(age_data)

# Ajouter le sexe encodé si disponible
if 'sex' in df.columns:
    sex_encoded = df['sex'].map({'male': 0, 'female': 1, np.nan: 0.5}).fillna(0.5).values.reshape(-1, 1)
    pca_features.append(sex_encoded)

# Ajouter les sites anatomiques encodés si disponibles
if 'anatom_site_general' in df.columns:
    site_dummies = pd.get_dummies(df['anatom_site_general'].fillna('unknown'), prefix='site')
    pca_features.append(site_dummies.values)

# Ajouter les labels de classe
pca_features.append(df[classes].values)

# Concaténer toutes les features
if pca_features:
    X_pca = np.hstack(pca_features)

    # Standardiser les données
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_pca)

    # Appliquer PCA
    n_components = min(5, X_scaled.shape[1])
    pca = PCA(n_components=n_components)
    X_pca_result = pca.fit_transform(X_scaled)

    print(f"\n📈 VARIANCE EXPLIQUÉE PAR COMPOSANTE :")
    for i, var in enumerate(pca.explained_variance_ratio_, 1):
        print(f"  PC{i}: {var:.3f} ({var*100:.1f}%)")

    print(f"📊 VARIANCE CUMULÉE : {pca.explained_variance_ratio_.cumsum()[-1]:.3f}")

    # Visualisation PCA
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))

    # 1. Variance expliquée
    components = range(1, len(pca.explained_variance_ratio_) + 1)
    axes[0, 0].bar(components, pca.explained_variance_ratio_, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].plot(components, pca.explained_variance_ratio_.cumsum(),
                    marker='o', color='red', linewidth=2, label='Cumulative')
    axes[0, 0].set_xlabel('Composantes Principales')
    axes[0, 0].set_ylabel('Variance Expliquée')
    axes[0, 0].set_title('Variance Expliquée par Composante', fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)

    # 2. Projection 2D (PC1 vs PC2) colorée par diagnostic
    # Trouver la classe principale pour chaque échantillon
    primary_classes = df[classes].idxmax(axis=1)

    scatter = axes[0, 1].scatter(X_pca_result[:, 0], X_pca_result[:, 1],
                                 c=pd.Categorical(primary_classes).codes,
                                 cmap='tab20', alpha=0.6, s=20)
    axes[0, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
    axes[0, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
    axes[0, 1].set_title('Projection 2D - Coloré par Diagnostic', fontweight='bold')

    # Créer une légende personnalisée
    legend_elements = []
    for i, cls in enumerate(classes[:8]):  # Limiter aux 8 premières classes
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w',
                                         markerfacecolor=plt.cm.tab20(i/8),
                                         markersize=8, label=cls))
    axes[0, 1].legend(handles=legend_elements, fontsize=8, loc='upper right')

    # 3. Contribution des features aux composantes
    # Calculer l'importance des features
    n_total_features = X_scaled.shape[1]
    feature_names = []

    # Construire les noms des features
    if 'age_approx' in df.columns:
        feature_names.append('Age')
    if 'sex' in df.columns:
        feature_names.append('Sex')
    if 'anatom_site_general' in df.columns:
        site_dummy_cols = site_dummies.columns.tolist()
        feature_names.extend([col.replace('site_', '').replace('_', ' ') for col in site_dummy_cols])
    feature_names.extend(classes)

    # Prendre les 15 features les plus importantes pour PC1
    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    feature_importance = pd.DataFrame(
        loadings[:, :2],
        index=feature_names[:n_total_features],
        columns=['PC1', 'PC2']
    ).abs().sort_values('PC1', ascending=False)

    feature_importance.head(10).plot(kind='bar', ax=axes[1, 0], color=['skyblue', 'lightcoral'])
    axes[1, 0].set_title('Top 10 Features Contribuant aux 2 Premières PC', fontweight='bold')
    axes[1, 0].set_ylabel('Contribution (valeur absolue)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(alpha=0.3)

    # 4. Biplot simplifié (PC1 vs PC2 avec vecteurs)
    axes[1, 1].scatter(X_pca_result[:, 0], X_pca_result[:, 1], alpha=0.3, s=10)

    # Ajouter les vecteurs pour les features importantes
    important_features = feature_importance.head(5).index.tolist()
    for i, feature in enumerate(important_features):
        if i < len(pca.components_[0]):
            axes[1, 1].arrow(0, 0, pca.components_[0, i]*3, pca.components_[1, i]*3,
                            head_width=0.05, head_length=0.05, fc='red', alpha=0.5)
            axes[1, 1].text(pca.components_[0, i]*3.2, pca.components_[1, i]*3.2,
                           feature, color='red', fontsize=9)

    axes[1, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
    axes[1, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
    axes[1, 1].set_title('Biplot Simplifié - Vecteurs des Features Importantes', fontweight='bold')
    axes[1, 1].grid(alpha=0.3)
    axes[1, 1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
    axes[1, 1].axvline(x=0, color='k', linestyle='-', alpha=0.3)

    plt.suptitle('Analyse en Composantes Principales (PCA) - ISIC 2019',
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(EDA_DIR / 'plots' / '07_pca_analysis_medical.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Analyse des clusters dans l'espace PCA
    print("\n🎯 CLUSTERS DANS L'ESPACE PCA :")

    # Utiliser les 3 premières composantes pour le clustering
    X_cluster = X_pca_result[:, :3]

    # Méthode du coude pour déterminer le nombre optimal de clusters
    inertias = []
    K_range = range(2, 11)

    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_cluster)
        inertias.append(kmeans.inertia_)

    # Trouver le point de coude
    diffs = np.diff(inertias)
    diff_diffs = np.diff(diffs)
    if len(diff_diffs) > 0:
        elbow_point = np.argmax(diff_diffs) + 3  # +3 car nous avons commencé à k=2
        optimal_k = min(max(3, elbow_point), 8)  # Limiter entre 3 et 8
        print(f"  Point de coude suggéré : {optimal_k} clusters")

        # Appliquer K-means avec k optimal
        kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(X_cluster)

        # Analyser la composition des clusters
        df['pca_cluster'] = cluster_labels

        print(f"\n📊 COMPOSITION DES {optimal_k} CLUSTERS PCA :")
        for cluster_id in range(optimal_k):
            cluster_data = df[df['pca_cluster'] == cluster_id]
            cluster_size = len(cluster_data)

            print(f"\n  Cluster {cluster_id} ({cluster_size} lésions, {cluster_size/len(df)*100:.1f}%) :")

            # Distribution des classes dans le cluster
            class_dist = cluster_data[classes].sum().sort_values(ascending=False)
            top_classes = class_dist.head(3)

            for cls, count in top_classes.items():
                percentage = count / cluster_size * 100
                if percentage > 10:  # Seuil de 10%
                    print(f"    • {cls} : {count} ({percentage:.1f}%)")

            # Caractéristiques démographiques si disponibles
            if 'age_approx' in df.columns:
                avg_age = cluster_data['age_approx'].mean()
                print(f"    • Âge moyen : {avg_age:.1f} ans")

            if 'sex' in df.columns:
                sex_dist = cluster_data['sex'].value_counts()
                if len(sex_dist) > 0:
                    print(f"    • Sexe : {dict(sex_dist)}")

        # Nettoyer
        df.drop('pca_cluster', axis=1, inplace=True)

    # Sauvegarder les résultats PCA
    pca_stats = {
        'explained_variance_ratio': pca.explained_variance_ratio_.tolist(),
        'cumulative_variance': pca.explained_variance_ratio_.cumsum().tolist(),
        'n_components': n_components,
        'optimal_clusters_suggested': int(optimal_k) if 'optimal_k' in locals() else None,
        'features_used': feature_names[:n_total_features],
        'pca_timestamp': datetime.now().isoformat()
    }

    with open(EDA_DIR / 'statistics' / '07_pca_analysis.json', 'w') as f:
        json.dump(pca_stats, f, indent=2)

    print(f"\n✅ Analyse PCA sauvegardée")

else:
    print("⚠️ Pas assez de métadonnées pour l'analyse PCA")

# #  7. CLUSTERING SUR LES CARACTÉRISTIQUES D'IMAGES

# %%
print("\n🎯 CLUSTERING SUR LES CARACTÉRISTIQUES VISUELLES")

def extract_image_features(image_path):
    """
    Extrait des caractéristiques simples d'une image
    """
    try:
        img = cv2.imread(str(image_path))
        if img is None:
            return None

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Caractéristiques de couleur
        mean_color = img.mean(axis=(0, 1))
        std_color = img.std(axis=(0, 1))

        # Caractéristiques de texture (simplifiées)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

        texture_magnitude = np.sqrt(sobelx**2 + sobely**2)
        texture_mean = texture_magnitude.mean()
        texture_std = texture_magnitude.std()

        # Caractéristiques de forme (rapport d'aspect)
        height, width = img.shape[:2]
        aspect_ratio = width / height

        # Combiner toutes les caractéristiques
        features = np.concatenate([
            mean_color,
            std_color,
            [texture_mean, texture_std, aspect_ratio]
        ])

        return features
    except:
        return None

# Extraire les caractéristiques d'un échantillon d'images
print(f"Extraction des caractéristiques visuelles... (échantillon de {min(500, len(df))} images)")

sample_size_clustering = min(500, len(df))
clustering_sample = df.sample(sample_size_clustering, random_state=42)

image_features = []
valid_indices = []

for idx, row in clustering_sample.iterrows():
    # Essayer avec .jpg
    img_path = IMG_DIR / f"{row['image']}.jpg"

    # Si pas trouvé, essayer sans extension
    if not img_path.exists():
        img_path = IMG_DIR / row['image']

    if img_path.exists():
        features = extract_image_features(img_path)
        if features is not None:
            image_features.append(features)
            valid_indices.append(idx)

if len(image_features) > 50:
    image_features = np.array(image_features)
    print(f"\n✅ {len(image_features)} images analysées avec succès")

    # Standardiser les caractéristiques
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(image_features)

    # Clustering avec K-means
    inertias = []
    silhouette_scores = []
    K_range = range(2, 11)  # k de 2 à 10

    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(features_scaled)
        inertias.append(kmeans.inertia_)

        # Calculer le score silhouette pour TOUS les k (y compris k=2)
        try:
            silhouette_avg = silhouette_score(features_scaled, cluster_labels)
            silhouette_scores.append(silhouette_avg)
        except Exception as e:
            print(f"⚠️ Erreur silhouette pour k={k}: {e}")
            silhouette_scores.append(0)

    print(f"\n📊 Scores silhouette calculés: {len(silhouette_scores)} valeurs")
    print(f"   K_range: {len(K_range)} valeurs")

    # Vérifier la cohérence des dimensions
    if len(silhouette_scores) != len(K_range):
        print(f"⚠️ Incohérence: silhouette_scores ({len(silhouette_scores)}) != K_range ({len(K_range)})")
        # Ajuster pour avoir la même longueur
        min_len = min(len(silhouette_scores), len(K_range))
        silhouette_scores = silhouette_scores[:min_len]
        K_range = list(K_range)[:min_len]

    # Visualisation
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # 1. Méthode du coude
    axes[0].plot(K_range, inertias, 'bo-', linewidth=2)
    axes[0].set_xlabel('Nombre de Clusters (k)')
    axes[0].set_ylabel('Inertie')
    axes[0].set_title('Méthode du Coude', fontweight='bold')
    axes[0].grid(alpha=0.3)

    # 2. Score silhouette
    if silhouette_scores and len(silhouette_scores) == len(K_range):
        axes[1].plot(K_range, silhouette_scores, 'ro-', linewidth=2)
        axes[1].set_xlabel('Nombre de Clusters (k)')
        axes[1].set_ylabel('Score Silhouette')
        axes[1].set_title('Score Silhouette', fontweight='bold')
        axes[1].grid(alpha=0.3)

        # Trouver le k optimal par silhouette
        optimal_k_silhouette = K_range[np.argmax(silhouette_scores)]
        print(f"🎯 Nombre optimal de clusters (silhouette) : {optimal_k_silhouette}")
        print(f"   Score silhouette max : {max(silhouette_scores):.3f}")

        # 3. Méthode du coude pour trouver k optimal
        # Calculer les différences secondes
        diffs = np.diff(inertias)
        if len(diffs) > 1:
            diff_diffs = np.diff(diffs)
            if len(diff_diffs) > 0:
                elbow_point = np.argmax(diff_diffs) + 2  # +2 car nous avons commencé à k=2
                optimal_k_elbow = min(max(3, elbow_point), 8)  # Limiter entre 3 et 8
                print(f"🎯 Nombre optimal de clusters (coude) : {optimal_k_elbow}")

                # Choisir le k optimal (silhouette si bon score, sinon coude)
                if max(silhouette_scores) > 0.2:  # Seuil raisonnable pour silhouette
                    optimal_k = optimal_k_silhouette
                    method = "silhouette"
                else:
                    optimal_k = optimal_k_elbow
                    method = "coude"

                print(f"🎯 K optimal final ({method}) : {optimal_k}")

                # Visualisation des clusters avec t-SNE
                tsne = TSNE(n_components=2, random_state=42, perplexity=30)
                features_2d = tsne.fit_transform(features_scaled)

                # Appliquer K-means avec k optimal
                kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
                final_clusters = kmeans_final.fit_predict(features_scaled)

                scatter = axes[2].scatter(features_2d[:, 0], features_2d[:, 1],
                                         c=final_clusters, cmap='tab20', s=20, alpha=0.6)
                axes[2].set_xlabel('t-SNE 1')
                axes[2].set_ylabel('t-SNE 2')
                axes[2].set_title(f'Clusters Visuels (k={optimal_k}, {method})', fontweight='bold')
                plt.colorbar(scatter, ax=axes[2], label='Cluster')

                # Sauvegarder les clusters pour analyse
                clustering_sample_filtered = clustering_sample.loc[valid_indices]
                clustering_sample_filtered['visual_cluster'] = final_clusters

                # Analyser les clusters
                print(f"\n📊 ANALYSE DES CLUSTERS VISUELS (k={optimal_k}):")

                for cluster_id in range(optimal_k):
                    cluster_data = clustering_sample_filtered[clustering_sample_filtered['visual_cluster'] == cluster_id]
                    cluster_size = len(cluster_data)

                    if cluster_size > 0:
                        print(f"\n  Cluster {cluster_id} ({cluster_size} images, {cluster_size/len(clustering_sample_filtered)*100:.1f}%) :")

                        # Distribution des classes
                        class_dist = cluster_data[classes].sum().sort_values(ascending=False)
                        top_classes = class_dist.head(2)

                        for cls, count in top_classes.items():
                            if count > 0:
                                percentage = count / cluster_size * 100
                                print(f"    • {cls} : {count} ({percentage:.1f}%)")

                        # Caractéristiques moyennes
                        cluster_features = image_features[final_clusters == cluster_id]
                        if len(cluster_features) > 0:
                            mean_color = cluster_features[:, :3].mean(axis=0)
                            print(f"    • Couleur moyenne : RGB({mean_color[0]:.0f}, {mean_color[1]:.0f}, {mean_color[2]:.0f})")
            else:
                axes[2].text(0.5, 0.5, 'Méthode du coude\nnon disponible',
                            ha='center', va='center', fontsize=12)
                axes[2].set_title('Visualisation Clusters', fontweight='bold')
                optimal_k = None
        else:
            axes[2].text(0.5, 0.5, 'Données insuffisantes\npour clustering',
                        ha='center', va='center', fontsize=12)
            axes[2].set_title('Visualisation Clusters', fontweight='bold')
            optimal_k = None
    else:
        axes[1].text(0.5, 0.5, 'Scores silhouette\nnon disponibles',
                    ha='center', va='center', fontsize=12)
        axes[1].set_title('Score Silhouette', fontweight='bold')

        axes[2].text(0.5, 0.5, 'Clustering non disponible\n(silhouette manquant)',
                    ha='center', va='center', fontsize=12)
        axes[2].set_title('Visualisation Clusters', fontweight='bold')
        optimal_k = None

    plt.suptitle('Clustering des Caractéristiques Visuelles', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(EDA_DIR / 'plots' / '08_visual_clustering.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Sauvegarder les résultats de clustering
    clustering_stats = {
        'samples_analyzed': len(image_features),
        'feature_dimension': image_features.shape[1],
        'k_range': [int(k) for k in K_range],
        'inertias': [float(i) for i in inertias],
        'silhouette_scores': [float(s) for s in silhouette_scores] if silhouette_scores else [],
        'clustering_timestamp': datetime.now().isoformat()
    }

    # Ajouter les k optimaux si calculés
    if 'optimal_k_silhouette' in locals():
        clustering_stats['optimal_k_silhouette'] = int(optimal_k_silhouette)
        clustering_stats['max_silhouette_score'] = float(max(silhouette_scores))

    if 'optimal_k_elbow' in locals():
        clustering_stats['optimal_k_elbow'] = int(optimal_k_elbow)

    if 'optimal_k' in locals() and optimal_k is not None:
        clustering_stats['optimal_k_final'] = int(optimal_k)
        clustering_stats['method_used'] = method if 'method' in locals() else 'unknown'

    with open(EDA_DIR / 'statistics' / '08_visual_clustering.json', 'w') as f:
        json.dump(clustering_stats, f, indent=2)

    print(f"\n✅ Clustering visuel sauvegardé")

else:
    print("⚠️ Pas assez d'images pour le clustering visuel")
    print(f"   Images analysées: {len(image_features)} (minimum requis: 50)")

# #  8. RAPPORT SYNTHÈSE DE L'ANALYSE AVANCÉE

# %%
print("\n" + "=" * 80)
print("   📊 RAPPORT SYNTHÈSE - ANALYSE AVANCÉE COMPLÈTE")
print("=" * 80)

# Créer un rapport détaillé
advanced_report = f"""# 📊 RAPPORT D'ANALYSE AVANCÉE - ISIC 2019

**Date d'analyse :** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Dataset :** ISIC 2019 (Skin Lesion Analysis)
**Images analysées :** {len(df):,} (70.1% du total)
**Classes diagnostiques :** {len(classes)}

## 🎨 1. ANALYSE DES COULEURS DES LÉSIONS

### Caractéristiques chromatiques globales
"""

if len(dominant_colors) > 0:
    advanced_report += f"""- **Couleur dominante moyenne :** RGB({dominant_colors[:, 0].mean():.0f}, {dominant_colors[:, 1].mean():.0f}, {dominant_colors[:, 2].mean():.0f})
- **Variabilité chromatique :** ±{dominant_colors.std(axis=0).mean():.1f} points RGB
- **Dominance de couleur :** {color_percentages.mean():.1f}% en moyenne

### Variations par diagnostic
"""

if 'color_stats' in locals() and 'class_stats' in color_stats and color_stats['class_stats']:
    for cls in classes[:3]:  # Limiter aux 3 premières classes
        if cls in color_stats['class_stats']['R']:
            r = color_stats['class_stats']['R'][cls]
            g = color_stats['class_stats']['G'][cls]
            b = color_stats['class_stats']['B'][cls]
            advanced_report += f"- **{cls} :** RGB({r:.0f}, {g:.0f}, {b:.0f})\n"
else:
    advanced_report += "- Données insuffisantes pour l'analyse par classe\n"

advanced_report += f"""
## 🔗 2. CORRÉLATIONS MÉTADONNÉES-CLASSES

### Corrélations significatives identifiées
"""

if 'significant_correlations' in locals() and significant_correlations:
    for metadata, cls, corr in significant_correlations[:5]:
        metadata_name = metadata.replace('site_', '').replace('_', ' ').title()
        direction = "positive" if corr > 0 else "négative"
        advanced_report += f"- **{metadata_name} ↔ {cls} :** r = {corr:.3f} ({direction})\n"
else:
    advanced_report += "- Aucune corrélation forte détectée\n"

advanced_report += f"""
### Implications cliniques
1. **Âge** : Corrélé avec certains types de lésions
2. **Localisation anatomique** : Influence le type de lésion
3. **Sexe** : Distribution différente selon les diagnostics

## 📊 3. ANALYSE MULTIDIMENSIONNELLE (PCA)

### Séparabilité des classes
"""

# Vérifier et formater les données PCA
pca_variance = "N/A"
pca_clusters = "N/A"
pca_features_list = "N/A"

if 'pca_stats' in locals():
    try:
        # Vérifier si explained_variance_ratio existe et a au moins 3 éléments
        if ('explained_variance_ratio' in pca_stats and
            pca_stats['explained_variance_ratio'] and
            len(pca_stats['explained_variance_ratio']) >= 3):

            # Convertir en numpy array pour utiliser sum()
            variance_array = np.array(pca_stats['explained_variance_ratio'][:3])
            pca_variance = f"{variance_array.sum()*100:.1f}"
        else:
            pca_variance = "Données insuffisantes"
    except Exception as e:
        pca_variance = f"Erreur: {str(e)[:50]}..."

    # Clusters suggérés
    if 'optimal_clusters_suggested' in pca_stats and pca_stats['optimal_clusters_suggested']:
        pca_clusters = str(pca_stats['optimal_clusters_suggested'])

    # Features importantes
    if 'feature_importance' in locals() and not feature_importance.empty:
        top_features = feature_importance.head(3).index.tolist()
        pca_features_list = ', '.join(top_features)

advanced_report += f"""- **Variance expliquée (3 premières PC) :** {pca_variance}%
- **Nombre de clusters suggérés :** {pca_clusters}
- **Features les plus discriminantes :** {pca_features_list}

## 🎯 4. CLUSTERING VISUEL

### Groupes naturels identifiés
"""

# Vérifier et formater les données de clustering
clustering_optimal_k = "N/A"
clustering_silhouette = "N/A"
clustering_samples = "N/A"

if 'clustering_stats' in locals():
    try:
        if 'optimal_k_final' in clustering_stats and clustering_stats['optimal_k_final']:
            clustering_optimal_k = str(clustering_stats['optimal_k_final'])

        if 'max_silhouette_score' in clustering_stats and clustering_stats['max_silhouette_score']:
            clustering_silhouette = f"{clustering_stats['max_silhouette_score']:.3f}"

        if 'samples_analyzed' in clustering_stats:
            clustering_samples = f"{clustering_stats['samples_analyzed']:,}"
    except:
        pass

advanced_report += f"""- **Clusters optimaux :** {clustering_optimal_k}
- **Score silhouette moyen :** {clustering_silhouette}
- **Images analysées :** {clustering_samples}

## 🏥 5. IMPLICATIONS POUR LE DIAGNOSTIC ASSISTÉ

### Patterns identifiés
1. **Signature chromatique** : Chaque diagnostic a un profil de couleur distinct
2. **Corrélations démographiques** : Âge et sexe influencent certains diagnostics
3. **Clusters visuels** : Groupes naturels correspondant partiellement aux diagnostics médicaux

### Recommandations pour le modèle
1. **Features d'entrée** : Inclure les métadonnées avec les images
2. **Attention aux biais** : Considérer les corrélations âge/sexe/localisation
3. **Augmentation spécifique** : Basée sur les clusters visuels identifiés

## ⚠️ 6. LIMITATIONS ET PRÉCAUTIONS

### Limitations techniques
1. **Dataset incomplet** : 70.1% des images seulement
"""

# Ajouter les statistiques de déséquilibre si disponibles
if 'class_stats' in locals() and 'imbalance_ratio' in class_stats:
    advanced_report += f"""2. **Déséquilibre prononcé** : Ratio {class_stats['imbalance_ratio']:.1f}:1 entre classes
3. **Métadonnées manquantes** : Variables cliniques partielles
"""
else:
    advanced_report += """2. **Déséquilibre prononcé** : Présent dans le dataset
3. **Métadonnées manquantes** : Variables cliniques partielles
"""

advanced_report += f"""
### Précautions cliniques
1. **Interprétation prudente** : Les corrélations ne sont pas des causalités
2. **Validation externe nécessaire** : Résultats à valider sur d'autres populations
3. **Considérations éthiques** : Biais potentiels dans les données

## 🚀 7. PROCHAINES ÉTAPES - MODÈLE MULTI-TÂCHES

### Architecture recommandée
1. **Backbone** : EfficientNet ou ResNet pré-entraîné
2. **Tâches multiples** :
   - Classification diagnostique principale
   - Prédiction des métadonnées (âge, sexe, localisation)
   - Segmentation des lésions (optionnelle)
3. **Loss adaptative** : Poids selon l'importance clinique

### Stratégie d'entraînement
1. **Transfer learning** : Fine-tuning à partir de modèles médicaux
2. **Augmentation** : Spécifique aux lésions cutanées
3. **Validation** : Cross-validation stratifiée
4. **Métriques** : AUC-ROC, Sensibilité, Spécificité, F1-Score

### Déploiement potentiel
1. **Interface** : Web-based avec upload d'images
2. **Explicabilité** : Cartes d'attention Grad-CAM
3. **Conformité** : Respect des régulations médicales

---

*Rapport généré automatiquement - Analyse avancée ISIC 2019*
*Prêt pour la phase de modélisation multi-tâches*
"""

# Sauvegarder le rapport
advanced_report_path = EDA_DIR / 'reports' / 'ADVANCED_ANALYSIS_REPORT.md'
with open(advanced_report_path, 'w', encoding='utf-8') as f:
    f.write(advanced_report)

print(f"\n✅ RAPPORT AVANCÉ GÉNÉRÉ ET SAUVEGARDÉ :")
print(f"    {advanced_report_path}")

# Afficher un résumé
print("\n" + "=" * 80)
print("   📊 RÉSUMÉ DES PRINCIPALES DÉCOUVERTES")
print("=" * 80)

print(f"\n📁 DONNÉES :")
print(f"  • Images totales : {len(df):,} (70.1% du dataset)")
print(f"  • Classes diagnostiques : {len(classes)}")
if 'class_stats' in locals() and 'imbalance_ratio' in class_stats:
    print(f"  • Déséquilibre max : {class_stats['imbalance_ratio']:.1f}:1")

print(f"\n🎨 COULEURS :")
if len(dominant_colors) > 0:
    print(f"  • Palette dominante : RGB({dominant_colors[:, 0].mean():.0f}, {dominant_colors[:, 1].mean():.0f}, {dominant_colors[:, 2].mean():.0f})")
    if 'color_stats' in locals() and 'class_stats' in color_stats and color_stats['class_stats']:
        print(f"  • Variations diagnostiques : Identifiées")
    else:
        print(f"  • Variations diagnostiques : Non disponibles")
else:
    print(f"  • Analyse couleur : Non disponible")

print(f"\n🔗 CORRÉLATIONS :")
if 'significant_correlations' in locals() and significant_correlations:
    top_corr = significant_correlations[0]
    metadata_name = top_corr[0].replace('site_', '').replace('_', ' ').title()
    print(f"  • Plus forte corrélation : {metadata_name} ↔ {top_corr[1]} (r={top_corr[2]:.3f})")
else:
    print(f"  • Corrélations : Faibles ou non significatives")

print(f"\n📊 PCA :")
if 'pca_stats' in locals():
    try:
        if ('explained_variance_ratio' in pca_stats and
            pca_stats['explained_variance_ratio'] and
            len(pca_stats['explained_variance_ratio']) >= 3):
            variance_array = np.array(pca_stats['explained_variance_ratio'][:3])
            print(f"  • Variance expliquée (3 PC) : {variance_array.sum()*100:.1f}%")
    except:
        pass

    if 'optimal_clusters_suggested' in pca_stats and pca_stats['optimal_clusters_suggested']:
        print(f"  • Clusters suggérés : {pca_stats['optimal_clusters_suggested']}")

print(f"\n🎯 CLUSTERING VISUEL :")
if 'clustering_stats' in locals():
    if 'optimal_k_final' in clustering_stats and clustering_stats['optimal_k_final']:
        print(f"  • Clusters optimaux : {clustering_stats['optimal_k_final']}")

    if 'max_silhouette_score' in clustering_stats and clustering_stats['max_silhouette_score']:
        print(f"  • Score silhouette : {clustering_stats['max_silhouette_score']:.3f}")

print(f"\n" + "=" * 80)
print("  ✅ ANALYSE EXPLORATOIRE AVANCÉE TERMINÉE !")
print("=" * 80)

# Sauvegarder toutes les données d'analyse avancée
print(f"\n💾 Sauvegarde des données d'analyse avancée...")

advanced_analysis_data = {
    'timestamp': datetime.now().isoformat(),
    'dataset_summary': {
        'total_analyzed': len(df),
        'classes': classes
    }
}

# Ajouter les statistiques de classe si disponibles
if 'class_stats' in locals():
    if 'coverage_percentage' in class_stats:
        advanced_analysis_data['dataset_summary']['coverage_percentage'] = class_stats['coverage_percentage']
    if 'imbalance_ratio' in class_stats:
        advanced_analysis_data['dataset_summary']['imbalance_ratio'] = class_stats['imbalance_ratio']

# Ajouter les données de couleur si disponibles
if len(dominant_colors) > 0:
    advanced_analysis_data['color_analysis'] = {
        'dominant_colors_mean': dominant_colors.mean(axis=0).tolist(),
        'dominant_colors_std': dominant_colors.std(axis=0).tolist(),
        'samples_analyzed': len(dominant_colors)
    }

# Ajouter les corrélations si disponibles
if 'significant_correlations' in locals() and significant_correlations:
    advanced_analysis_data['correlation_analysis'] = {
        'top_correlations': [
            {'metadata': m, 'class': c, 'correlation': float(corr)}
            for m, c, corr in significant_correlations[:10]
        ]
    }

# Ajouter les statistiques PCA si disponibles
if 'pca_stats' in locals():
    advanced_analysis_data['pca_analysis'] = pca_stats

# Ajouter les statistiques de clustering si disponibles
if 'clustering_stats' in locals():
    advanced_analysis_data['visual_clustering'] = clustering_stats

# Sauvegarder
with open(EDA_DIR / 'statistics' / 'advanced_analysis_summary.json', 'w') as f:
    json.dump(advanced_analysis_data, f, indent=2)

print(f"✅ Données d'analyse avancée sauvegardées")

print(f"\n" + "=" * 80)
print("    ANALYSE COMPLÈTE TERMINÉE AVEC SUCCÈS !")
print("=" * 80)

print(f"\n📋 PROCHAINE ÉTAPE : PRÉTRAITEMENT POUR LE MODÈLE MULTI-TÂCHES")
print(f"\n1. ✅ EDA de base terminée")
print(f"2. ✅ Analyse avancée terminée")
print(f"3. ⏳ Prêt pour le prétraitement et la modélisation")
print(f"\n📂 Tous les résultats sont dans : {EDA_DIR}")
print("=" * 80)

In [ ]:
# =============================================================================
# PRÉTRAITEMENT ISIC 2019 – STANDARD MÉDICAL PROFESSIONNEL
# Conforme aux exigences du domaine dermatologique et imagerie médicale
# =============================================================================

import os, cv2, numpy as np, pandas as pd, json, hashlib, zipfile
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import albumentations as A
from sklearn.model_selection import StratifiedKFold
from google.colab import files
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION MÉDICALE PROFESSIONNELLE
# =============================================================================

class MedicalPreprocessConfig:
    """Configuration validée pour l'imagerie dermatoscopique"""

    # Standards d'imagerie dermatologique
    TARGET_SIZE = 384  # Résolution validée pour analyse dermatologique
    INTERPOLATION = cv2.INTER_LANCZOS4  # Meilleure qualité pour imagerie médicale

    # Normalisation ImageNet (standard pour transfer learning médical)
    MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)

    # Paramètres de qualité et validation
    MIN_IMAGE_SIZE = 50  # Taille minimale acceptable
    MAX_ASPECT_RATIO = 3.0  # Ratio max hauteur/largeur
    QUALITY_THRESHOLD = 10  # Seuil variance pour détecter images corrompues

    # Gestion des classes déséquilibrées (approche conservatrice)
    RARE_CLASSES = ['DF', 'VASC', 'AK', 'SCC']
    AUGMENTATION_FACTOR = 4  # Facteur réduit pour éviter l'overfitting

    # Chemins (à adapter selon votre environnement)
    BASE_DIR = Path('/content/drive/MyDrive/ISIC_2019_Project/data')
    IMG_DIR = BASE_DIR / 'ISIC_2019_Training_Input/ISIC_2019_Training_Input'
    OUTPUT_ROOT = Path('/content/ISIC2019_Medical_Professional')
    VISUALIZATION_DIR = OUTPUT_ROOT / 'visualizations'

CONFIG = MedicalPreprocessConfig()

# =============================================================================
# PIPELINE DE PRÉTRAITEMENT MÉDICAL
# =============================================================================

class MedicalImagePreprocessor:
    """
    Préprocesseur conforme aux standards d'imagerie dermatologique.

    Étapes validées:
    1. Validation qualité image
    2. Removal artefacts (poils) - méthode Dull Razor
    3. Normalisation couleur CLAHE adaptative
    4. Segmentation ROI (région d'intérêt)
    5. Padding et resize avec conservation aspect ratio
    6. Normalisation statistique
    """

    def __init__(self, config):
        self.config = config
        self.processing_log = []
        self.visualization_samples = []  # Pour stocker exemples avant/après

    def validate_image(self, img, img_name):
        """Validation qualité selon standards médicaux"""
        if img is None:
            self.log_issue(img_name, "Image non lisible")
            return False

        h, w = img.shape[:2]

        # Vérifications dimensionnelles
        if h < self.config.MIN_IMAGE_SIZE or w < self.config.MIN_IMAGE_SIZE:
            self.log_issue(img_name, f"Dimensions insuffisantes: {w}x{h}")
            return False

        aspect_ratio = max(h/w, w/h)
        if aspect_ratio > self.config.MAX_ASPECT_RATIO:
            self.log_issue(img_name, f"Aspect ratio anormal: {aspect_ratio:.2f}")
            return False

        # Détection images corrompues/uniformes
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        variance = np.var(gray)
        if variance < self.config.QUALITY_THRESHOLD:
            self.log_issue(img_name, f"Variance trop faible: {variance:.2f}")
            return False

        return True

    def remove_hair_artifacts(self, img):
        """
        Hair removal adapté - Méthode Dull Razor optimisée
        Référence: Lee et al. "DullRazor: A Software Approach to Hair Removal"
        """
        # Conversion espace de travail
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Détection poils via morphologie
        kernel_size = max(15, int(min(img.shape[:2]) * 0.02))  # Adaptatif à la taille
        if kernel_size % 2 == 0:
            kernel_size += 1
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))

        # Black-hat transform pour isoler structures sombres fines
        blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

        # Seuillage adaptatif
        _, mask = cv2.threshold(blackhat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Inpainting conservatif
        result = cv2.inpaint(img, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

        return result

    def adaptive_color_normalization(self, img):
        """
        Normalisation couleur adaptative pour compenser variations d'éclairage.
        Utilise CLAHE en espace LAB (standard dermatologique)
        """
        # Conversion LAB (perceptuellement uniforme)
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)

        # CLAHE sur canal luminance
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
        l_enhanced = clahe.apply(l)

        # Reconstruction
        enhanced = cv2.merge([l_enhanced, a, b])
        result = cv2.cvtColor(enhanced, cv2.COLOR_LAB2RGB)

        return result

    def extract_roi_with_padding(self, img):
        """
        Extraction ROI (Region of Interest) avec padding intelligent.
        Élimine le fond noir tout en conservant la lésion complète.
        """
        # Détection du contenu
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        _, binary = cv2.threshold(gray, 15, 255, cv2.THRESH_BINARY)

        # Nettoyage morphologique
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

        # Extraction bounding box avec marge de sécurité
        coords = cv2.findNonZero(binary)
        if coords is not None:
            x, y, w, h = cv2.boundingRect(coords)

            # Marge de sécurité (5% de chaque côté)
            margin_w = int(w * 0.05)
            margin_h = int(h * 0.05)

            x = max(0, x - margin_w)
            y = max(0, y - margin_h)
            w = min(img.shape[1] - x, w + 2 * margin_w)
            h = min(img.shape[0] - y, h + 2 * margin_h)

            img = img[y:y+h, x:x+w]

        # Padding carré avec noir (préserve contexte médical)
        h, w = img.shape[:2]
        max_dim = max(h, w)

        # Canvas noir
        padded = np.zeros((max_dim, max_dim, 3), dtype=img.dtype)

        # Centrage
        y_offset = (max_dim - h) // 2
        x_offset = (max_dim - w) // 2
        padded[y_offset:y_offset+h, x_offset:x_offset+w] = img

        return padded

    def resize_preserve_quality(self, img, target_size):
        """Resize avec interpolation haute qualité"""
        return cv2.resize(img, (target_size, target_size),
                         interpolation=self.config.INTERPOLATION)

    def normalize_intensity(self, img):
        """Normalisation statistique finale"""
        img_float = img.astype(np.float32) / 255.0
        normalized = (img_float - self.config.MEAN) / self.config.STD
        return normalized

    def process_image(self, img_path, img_name, save_steps=False):
        """Pipeline complet de prétraitement avec option de sauvegarde étapes"""
        try:
            # Lecture
            img_original = cv2.imread(str(img_path))
            if img_original is None:
                return None, None
            img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)

            # Validation
            if not self.validate_image(img_original, img_name):
                return None, None

            # Stockage des étapes pour visualisation
            steps = {} if save_steps else None
            if save_steps:
                steps['original'] = img_original.copy()

            # Pipeline avec sauvegarde étapes
            img = img_original.copy()

            # Étape 1: Hair removal
            img = self.remove_hair_artifacts(img)
            if save_steps:
                steps['hair_removed'] = img.copy()

            # Étape 2: Normalisation couleur
            img = self.adaptive_color_normalization(img)
            if save_steps:
                steps['color_normalized'] = img.copy()

            # Étape 3: ROI extraction
            img = self.extract_roi_with_padding(img)
            if save_steps:
                steps['roi_extracted'] = img.copy()

            # Étape 4: Resize
            img = self.resize_preserve_quality(img, self.config.TARGET_SIZE)
            if save_steps:
                steps['resized'] = img.copy()

            # Étape 5: Normalisation finale
            img = self.normalize_intensity(img)
            if save_steps:
                steps['normalized'] = img

            return img, steps

        except Exception as e:
            self.log_issue(img_name, f"Erreur traitement: {str(e)}")
            return None, None

    def log_issue(self, img_name, issue):
        """Journalisation des problèmes pour traçabilité"""
        self.processing_log.append({
            'image': img_name,
            'issue': issue,
            'timestamp': datetime.now().isoformat()
        })

    def save_visualization_sample(self, img_name, steps, label):
        """Sauvegarde un échantillon pour visualisation"""
        self.visualization_samples.append({
            'image_name': img_name,
            'steps': steps,
            'label': label
        })

# =============================================================================
# FONCTIONS DE VISUALISATION
# =============================================================================

def create_processing_visualization(sample, output_path):
    """Crée une visualisation avant/après du pipeline"""
    steps = sample['steps']
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f"Pipeline de Prétraitement: {sample['image_name']} (Classe: {sample['label']})",
                 fontsize=16, fontweight='bold')

    # Ordre des étapes à afficher
    step_order = [
        ('original', 'Image Originale'),
        ('hair_removed', 'Removal Cheveux'),
        ('color_normalized', 'Normalisation Couleur (CLAHE)'),
        ('roi_extracted', 'Extraction ROI + Padding'),
        ('resized', 'Resize 384x384'),
        ('normalized', 'Normalisation Finale')
    ]

    for idx, (step_key, step_title) in enumerate(step_order):
        row = idx // 3
        col = idx % 3
        ax = axes[row, col]

        if step_key in steps:
            img = steps[step_key]

            # Dénormalisation si étape finale
            if step_key == 'normalized':
                img_display = img * CONFIG.STD + CONFIG.MEAN
                img_display = np.clip(img_display * 255, 0, 255).astype(np.uint8)
            else:
                img_display = img

            ax.imshow(img_display)
            ax.set_title(step_title, fontsize=12, fontweight='bold')
            ax.axis('off')

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

def create_distribution_plots(train_df, val_df, test_df, output_dir):
    """Crée les graphiques de distribution des classes"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Distribution des Classes - Dataset ISIC 2019', fontsize=16, fontweight='bold')

    # Palette de couleurs
    colors = sns.color_palette("Set2", 8)

    # 1. Distribution globale
    ax = axes[0, 0]
    all_data = pd.concat([train_df, val_df, test_df])
    class_counts = all_data['label'].value_counts().sort_index()
    ax.bar(range(len(class_counts)), class_counts.values, color=colors)
    ax.set_xticks(range(len(class_counts)))
    ax.set_xticklabels(class_counts.index, rotation=45)
    ax.set_title('Distribution Globale', fontweight='bold')
    ax.set_ylabel('Nombre d\'images')
    ax.grid(axis='y', alpha=0.3)

    # 2. Distribution par split
    ax = axes[0, 1]
    split_data = {
        'Train': train_df['label'].value_counts().sort_index(),
        'Val': val_df['label'].value_counts().sort_index(),
        'Test': test_df['label'].value_counts().sort_index()
    }
    x = np.arange(len(class_counts))
    width = 0.25
    for idx, (split_name, counts) in enumerate(split_data.items()):
        ax.bar(x + idx*width, counts.values, width, label=split_name, alpha=0.8)
    ax.set_xticks(x + width)
    ax.set_xticklabels(class_counts.index, rotation=45)
    ax.set_title('Distribution par Split', fontweight='bold')
    ax.set_ylabel('Nombre d\'images')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

    # 3. Proportions en camembert - Train
    ax = axes[1, 0]
    train_counts = train_df['label'].value_counts().sort_index()
    ax.pie(train_counts.values, labels=train_counts.index, autopct='%1.1f%%',
           colors=colors, startangle=90)
    ax.set_title('Proportions Train', fontweight='bold')

    # 4. Tableau récapitulatif
    ax = axes[1, 1]
    ax.axis('off')

    summary_data = []
    for label in class_counts.index:
        train_count = train_df[train_df['label'] == label].shape[0]
        val_count = val_df[val_df['label'] == label].shape[0]
        test_count = test_df[test_df['label'] == label].shape[0]
        total = train_count + val_count + test_count
        summary_data.append([label, train_count, val_count, test_count, total])

    table = ax.table(cellText=summary_data,
                     colLabels=['Classe', 'Train', 'Val', 'Test', 'Total'],
                     cellLoc='center',
                     loc='center',
                     bbox=[0, 0, 1, 1])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)

    # Style du header
    for i in range(5):
        table[(0, i)].set_facecolor('#4CAF50')
        table[(0, i)].set_text_props(weight='bold', color='white')

    ax.set_title('Tableau Récapitulatif', fontweight='bold', pad=20)

    plt.tight_layout()
    plt.savefig(output_dir / 'class_distribution.png', dpi=150, bbox_inches='tight')
    plt.close()

def create_quality_report_visualization(quality_report, processing_log, output_dir):
    """Crée une visualisation du rapport qualité"""
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

    fig.suptitle('Rapport Qualité du Prétraitement', fontsize=16, fontweight='bold')

    # 1. Statistiques générales
    ax1 = fig.add_subplot(gs[0, :])
    ax1.axis('off')

    stats_text = f"""
    📊 STATISTIQUES GÉNÉRALES

    Date de traitement: {quality_report['processing_date'][:10]}
    Images totales: {quality_report['total_images']}

    📁 RÉPARTITION
    Train: {quality_report['train_images']} images ({quality_report['train_images']/quality_report['total_images']*100:.1f}%)
    Validation: {quality_report['val_images']} images ({quality_report['val_images']/quality_report['total_images']*100:.1f}%)
    Test: {quality_report['test_images']} images ({quality_report['test_images']/quality_report['total_images']*100:.1f}%)

    ⚙️ PARAMÈTRES
    Taille cible: {quality_report['target_size']}x{quality_report['target_size']} pixels
    Facteur augmentation: x{quality_report['augmentation_factor']} (classes rares uniquement)
    Classes rares: {', '.join(quality_report['rare_classes'])}
    """

    ax1.text(0.1, 0.5, stats_text, fontsize=12, verticalalignment='center',
             fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

    # 2. Issues détectées
    ax2 = fig.add_subplot(gs[1, :])

    if len(processing_log) > 0:
        issue_types = {}
        for log in processing_log:
            issue = log['issue'].split(':')[0]
            issue_types[issue] = issue_types.get(issue, 0) + 1

        issues = list(issue_types.keys())
        counts = list(issue_types.values())

        colors_issues = sns.color_palette("Reds", len(issues))
        ax2.barh(issues, counts, color=colors_issues)
        ax2.set_xlabel('Nombre d\'occurrences')
        ax2.set_title(f'⚠️ Problèmes Détectés ({len(processing_log)} total)', fontweight='bold')
        ax2.grid(axis='x', alpha=0.3)
    else:
        ax2.text(0.5, 0.5, '✅ Aucun problème détecté\nToutes les images traitées avec succès',
                ha='center', va='center', fontsize=14, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
        ax2.axis('off')

    # 3. Pipeline de traitement
    ax3 = fig.add_subplot(gs[2, :])
    ax3.axis('off')

    pipeline_text = """
    🔬 PIPELINE DE PRÉTRAITEMENT MÉDICAL

    ✓ Validation qualité (dimensions, aspect ratio, variance)
    ✓ Removal artefacts cheveux (Méthode Dull Razor)
    ✓ Normalisation couleur adaptative (CLAHE en LAB)
    ✓ Extraction ROI avec marges de sécurité (5%)
    ✓ Padding intelligent et centrage
    ✓ Resize haute qualité (LANCZOS4)
    ✓ Normalisation statistique (ImageNet: μ={}, σ={})
    ✓ Augmentation conservatrice (géométrique uniquement)
    """.format(
        [f"{m:.3f}" for m in quality_report['normalization']['mean']],
        [f"{s:.3f}" for s in quality_report['normalization']['std']]
    )

    ax3.text(0.1, 0.5, pipeline_text, fontsize=11, verticalalignment='center',
             fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

    plt.savefig(output_dir / 'quality_report.png', dpi=150, bbox_inches='tight')
    plt.close()

# =============================================================================
# AUGMENTATION MÉDICALE CONSERVATRICE
# =============================================================================

def get_medical_augmentation_pipeline():
    """
    Augmentations validées pour imagerie dermatologique.
    Uniquement transformations géométriques préservant les caractéristiques cliniques.
    """
    return A.Compose([
        # Transformations géométriques (cliniquement neutres)
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.05,
            scale_limit=0.1,
            rotate_limit=15,
            border_mode=cv2.BORDER_CONSTANT,
            value=0,
            p=0.5
        ),

        # Variations photométriques légères (simulent conditions acquisition)
        A.RandomBrightnessContrast(
            brightness_limit=0.1,
            contrast_limit=0.1,
            p=0.3
        ),

        # Pas de déformations élastiques ni de modifications de couleur
        # qui altéreraient les caractéristiques diagnostiques
    ], p=0.8)

# =============================================================================
# STRATIFICATION ET SPLIT PROFESSIONNEL
# =============================================================================

def create_stratified_splits(df, config, n_folds=5, test_size=0.15):
    """
    Split stratifié avec validation croisée optionnelle.
    Garantit distribution équilibrée des classes dans tous les sets.
    """
    print("\n🔬 Création des splits stratifiés...")

    # Split initial test
    test_indices = []
    train_val_indices = []

    for label in df['label'].unique():
        label_indices = df[df['label'] == label].index.tolist()
        n_test = max(1, int(len(label_indices) * test_size))

        np.random.seed(42)
        test_idx = np.random.choice(label_indices, size=n_test, replace=False)
        train_val_idx = [i for i in label_indices if i not in test_idx]

        test_indices.extend(test_idx)
        train_val_indices.extend(train_val_idx)

    test_df = df.loc[test_indices].reset_index(drop=True)
    train_val_df = df.loc[train_val_indices].reset_index(drop=True)

    # Split train/val stratifié
    val_size = 0.15 / (1 - test_size)  # ~15% du total
    val_indices = []
    train_indices = []

    for label in train_val_df['label'].unique():
        label_indices = train_val_df[train_val_df['label'] == label].index.tolist()
        n_val = max(1, int(len(label_indices) * val_size))

        np.random.seed(42)
        val_idx = np.random.choice(label_indices, size=n_val, replace=False)
        train_idx = [i for i in label_indices if i not in val_idx]

        val_indices.extend(val_idx)
        train_indices.extend(train_idx)

    train_df = train_val_df.loc[train_indices].reset_index(drop=True)
    val_df = train_val_df.loc[val_indices].reset_index(drop=True)

    # Statistiques
    print(f"\n📊 Distribution des données:")
    print(f"   Train: {len(train_df)} images ({len(train_df)/len(df)*100:.1f}%)")
    print(f"   Val:   {len(val_df)} images ({len(val_df)/len(df)*100:.1f}%)")
    print(f"   Test:  {len(test_df)} images ({len(test_df)/len(df)*100:.1f}%)")

    # Vérification distribution par classe
    print(f"\n📋 Distribution par classe:")
    for split_name, split_df in [('Train', train_df), ('Val', val_df), ('Test', test_df)]:
        dist = split_df['label'].value_counts().sort_index()
        print(f"\n{split_name}:")
        for label, count in dist.items():
            print(f"   {label}: {count:4d} ({count/len(split_df)*100:5.1f}%)")

    return train_df, val_df, test_df

# =============================================================================
# PIPELINE PRINCIPAL
# =============================================================================

def main():
    """Pipeline principal de prétraitement médical"""

    print("="*80)
    print("PRÉTRAITEMENT ISIC 2019 - STANDARD MÉDICAL PROFESSIONNEL")
    print("="*80)
    print(f"\n⏰ Début: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Création structure de sortie
    output_dirs = {
        'images': CONFIG.OUTPUT_ROOT / 'images',
        'train': CONFIG.OUTPUT_ROOT / 'images' / 'train',
        'val': CONFIG.OUTPUT_ROOT / 'images' / 'val',
        'test': CONFIG.OUTPUT_ROOT / 'images' / 'test',
        'logs': CONFIG.OUTPUT_ROOT / 'logs',
        'metadata': CONFIG.OUTPUT_ROOT / 'metadata',
        'visualizations': CONFIG.VISUALIZATION_DIR
    }

    for dir_path in output_dirs.values():
        dir_path.mkdir(parents=True, exist_ok=True)

    # Chargement et fusion données
    print("\n📥 Chargement des métadonnées...")
    df = pd.read_csv(CONFIG.BASE_DIR / 'ISIC_2019_Training_GroundTruth.csv')
    meta = pd.read_csv(CONFIG.BASE_DIR / 'ISIC_2019_Training_Metadata.csv')
    df = pd.merge(df, meta, on='image', how='inner')

    # Validation chemins
    df['path'] = df['image'].apply(lambda x: CONFIG.IMG_DIR / f"{x}.jpg")
    df = df[df['path'].apply(lambda p: p.exists())].reset_index(drop=True)

    # Extraction labels
    label_cols = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']
    df['label'] = df[label_cols].idxmax(axis=1)

    print(f"✅ {len(df)} images valides chargées")
    print(f"\n🏥 Distribution initiale des classes:")
    for label, count in df['label'].value_counts().sort_index().items():
        print(f"   {label}: {count:5d} ({count/len(df)*100:.1f}%)")

    # Création splits stratifiés
    train_df, val_df, test_df = create_stratified_splits(df, CONFIG)

    # Initialisation préprocesseur
    preprocessor = MedicalImagePreprocessor(CONFIG)
    augmentor = get_medical_augmentation_pipeline()

    # Sélection d'échantillons pour visualisation (2 par classe)
    samples_to_visualize = []
    for label in df['label'].unique():
        label_samples = train_df[train_df['label'] == label].head(2)
        samples_to_visualize.extend(label_samples.index.tolist())

    print(f"\n🎨 {len(samples_to_visualize)} échantillons sélectionnés pour visualisation")

    # Traitement par split
    for split_name, split_df in [('train', train_df), ('val', val_df), ('test', test_df)]:
        print(f"\n{'='*80}")
        print(f"🔬 Traitement {split_name.upper()} ({len(split_df)} images)")
        print(f"{'='*80}")

        processed_count = 0
        augmented_count = 0

        for idx, row in tqdm(split_df.iterrows(), total=len(split_df),
                            desc=f"Processing {split_name}"):

            # Vérifier si on doit sauvegarder les étapes pour visualisation
            save_steps = (split_name == 'train' and idx in samples_to_visualize)

            # Image originale
            img_processed, steps = preprocessor.process_image(
                row['path'], row['image'], save_steps=save_steps
            )

            if img_processed is not None:
                output_path = output_dirs[split_name] / f"{row['image']}.npy"
                np.save(output_path, img_processed)
                processed_count += 1

                # Sauvegarde échantillon pour visualisation
                if save_steps and steps:
                    preprocessor.save_visualization_sample(row['image'], steps, row['label'])

                # Augmentation uniquement classes rares (train only)
                if split_name == 'train' and row['label'] in CONFIG.RARE_CLASSES:
                    # Dénormalisation pour augmentation
                    img_denorm = img_processed * CONFIG.STD + CONFIG.MEAN
                    img_uint8 = np.clip(img_denorm * 255, 0, 255).astype(np.uint8)

                    for aug_idx in range(CONFIG.AUGMENTATION_FACTOR):
                        aug_result = augmentor(image=img_uint8)
                        img_aug = aug_result['image']

                        # Re-normalisation
                        img_aug_norm = (img_aug.astype(np.float32) / 255.0 - CONFIG.MEAN) / CONFIG.STD

                        aug_path = output_dirs[split_name] / f"{row['image']}_aug{aug_idx}.npy"
                        np.save(aug_path, img_aug_norm)
                        augmented_count += 1

        print(f"\n✅ {split_name.upper()} terminé:")
        print(f"   - Images traitées: {processed_count}/{len(split_df)}")
        if augmented_count > 0:
            print(f"   - Images augmentées: {augmented_count}")

    # Sauvegarde métadonnées
    print("\n💾 Sauvegarde des métadonnées...")
    train_df.to_csv(output_dirs['metadata'] / 'train_metadata.csv', index=False)
    val_df.to_csv(output_dirs['metadata'] / 'val_metadata.csv', index=False)
    test_df.to_csv(output_dirs['metadata'] / 'test_metadata.csv', index=False)

    # Sauvegarde logs de traitement
    if preprocessor.processing_log:
        log_df = pd.DataFrame(preprocessor.processing_log)
        log_df.to_csv(output_dirs['logs'] / 'processing_issues.csv', index=False)
        print(f"⚠️  {len(preprocessor.processing_log)} problèmes détectés (voir logs)")

    # Génération rapport qualité
    quality_report = {
        'processing_date': datetime.now().isoformat(),
        'total_images': len(df),
        'train_images': len(train_df),
        'val_images': len(val_df),
        'test_images': len(test_df),
        'target_size': CONFIG.TARGET_SIZE,
        'normalization': {'mean': CONFIG.MEAN.tolist(), 'std': CONFIG.STD.tolist()},
        'augmentation_factor': CONFIG.AUGMENTATION_FACTOR,
        'rare_classes': CONFIG.RARE_CLASSES,
        'issues_detected': len(preprocessor.processing_log)
    }

    with open(output_dirs['metadata'] / 'quality_report.json', 'w') as f:
        json.dump(quality_report, f, indent=2)

    # GÉNÉRATION DES VISUALISATIONS
    print("\n" + "="*80)
    print("🎨 GÉNÉRATION DES VISUALISATIONS")
    print("="*80)

    # 1. Visualisations du pipeline (échantillons)
    print("\n📸 Création des visualisations pipeline...")
    for idx, sample in enumerate(tqdm(preprocessor.visualization_samples, desc="Pipeline viz")):
        output_file = output_dirs['visualizations'] / f'pipeline_{idx+1}_{sample["label"]}.png'
        create_processing_visualization(sample, output_file)

    print(f"✅ {len(preprocessor.visualization_samples)} visualisations pipeline créées")

    # 2. Graphiques de distribution
    print("\n📊 Création des graphiques de distribution...")
    create_distribution_plots(train_df, val_df, test_df, output_dirs['visualizations'])
    print("✅ Graphiques de distribution créés")

    # 3. Rapport qualité visuel
    print("\n📋 Création du rapport qualité visuel...")
    create_quality_report_visualization(
        quality_report,
        preprocessor.processing_log,
        output_dirs['visualizations']
    )
    print("✅ Rapport qualité visuel créé")

    # Création d'un README
    readme_content = f"""# ISIC 2019 - Dataset Prétraité (Standard Médical)

## 📊 Informations Générales

- **Date de traitement**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **Images totales**: {len(df)}
- **Train**: {len(train_df)} images
- **Validation**: {len(val_df)} images
- **Test**: {len(test_df)} images

## 🔬 Pipeline de Prétraitement

1. ✅ **Validation qualité** (dimensions, aspect ratio, variance)
2. ✅ **Removal artefacts cheveux** (Méthode Dull Razor)
3. ✅ **Normalisation couleur** (CLAHE adaptatif en LAB)
4. ✅ **Extraction ROI** avec marges de sécurité (5%)
5. ✅ **Padding intelligent** et centrage
6. ✅ **Resize haute qualité** (LANCZOS4 → {CONFIG.TARGET_SIZE}x{CONFIG.TARGET_SIZE})
7. ✅ **Normalisation statistique** (ImageNet mean/std)
8. ✅ **Augmentation conservatrice** (x{CONFIG.AUGMENTATION_FACTOR}, classes rares uniquement)

## 📁 Structure du Dataset

```
ISIC2019_Medical_Professional/
├── images/
│   ├── train/          # Images train (.npy format)
│   ├── val/            # Images validation (.npy format)
│   └── test/           # Images test (.npy format)
├── metadata/
│   ├── train_metadata.csv
│   ├── val_metadata.csv
│   ├── test_metadata.csv
│   └── quality_report.json
├── visualizations/
│   ├── pipeline_*.png          # Visualisations avant/après
│   ├── class_distribution.png  # Distribution des classes
│   └── quality_report.png      # Rapport qualité
├── logs/
│   └── processing_issues.csv   # Problèmes détectés (si applicable)
└── README.md
```

## 🏥 Classes (8 lésions dermatologiques)

- **MEL**: Melanoma
- **NV**: Melanocytic nevus
- **BCC**: Basal cell carcinoma
- **AK**: Actinic keratosis (rare - augmentée)
- **BKL**: Benign keratosis
- **DF**: Dermatofibroma (rare - augmentée)
- **VASC**: Vascular lesion (rare - augmentée)
- **SCC**: Squamous cell carcinoma (rare - augmentée)

## 💾 Utilisation

```python
import numpy as np

# Charger une image
img = np.load('images/train/ISIC_0000001.npy')

# Format: (384, 384, 3), dtype: float32
# Déjà normalisé avec mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
```

## ⚙️ Paramètres Techniques

- **Taille cible**: {CONFIG.TARGET_SIZE}x{CONFIG.TARGET_SIZE} pixels
- **Interpolation**: LANCZOS4 (haute qualité)
- **Normalisation**: ImageNet (mean/std)
- **Format**: NumPy .npy (float32)
- **Augmentation**: Géométrique uniquement (préserve caractéristiques cliniques)

## 📈 Visualisations Incluses

Consultez le dossier `visualizations/` pour:
- Exemples avant/après du pipeline complet
- Distribution des classes par split
- Rapport qualité détaillé

## ✅ Validation Qualité

- {len(preprocessor.processing_log)} problème(s) détecté(s)
- Voir `logs/processing_issues.csv` pour détails

---
**Standard médical professionnel | Conforme imagerie dermatoscopique**
"""

    with open(CONFIG.OUTPUT_ROOT / 'README.md', 'w', encoding='utf-8') as f:
        f.write(readme_content)

    # Compression et téléchargement vers machine locale
    print("\n📦 Compression du dataset...")
    zip_filename = 'ISIC2019_Medical_Professional.zip'
    zip_path_local = f'/content/{zip_filename}'

    # Compression
    os.system(f'cd /content && zip -r -q {zip_path_local} {CONFIG.OUTPUT_ROOT.name}')

    # Téléchargement vers votre machine locale
    print("\n📥 Téléchargement vers votre machine locale...")
    print("⏳ Le téléchargement va démarrer automatiquement...")
    files.download(zip_path_local)

    print(f"\n✅ Prétraitement terminé avec succès!")
    print(f"⏰ Fin: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"\n📊 Résumé:")
    print(f"   - Dataset prêt pour entraînement professionnel")
    print(f"   - Qualité médicale garantie")
    print(f"   - Traçabilité complète assurée")
    print(f"\n💾 Fichier ZIP:")
    print(f"   - ✅ Téléchargé sur votre machine locale")
    print(f"   - 📁 Vérifiez votre dossier 'Téléchargements'")

if __name__ == "__main__":
    main()

In [ ]:
from google.colab import files
import os

# Chemin du fichier zip dans Colab
source = '/content/ISIC2019_Medical_Professional.zip'

# Vérifier que le fichier existe
if os.path.exists(source):
    print("Téléchargement en cours...")
    files.download(source)
    print("✅ Téléchargement lancé ! Vérifiez le dossier 'Téléchargements' de votre ordinateur.")
else:
    print("❌ Fichier introuvable dans Colab :", source)